In [ ]:
from google.colab import drive
#drive.mount('/content/drive')
#drive.flush_and_unmount()
#!rm -rf /content/drive

drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install tf-keras-vis
# -*- coding: utf-8 -*-
"""GradCam.ipynb

Automatically generated by Colaboratory.

Original file is located at
    https://colab.research.google.com/drive/1cKjutZfO-CAwKnI4E84NAhm4seqoyGi0
"""

#!pip install tf-keras-vis

import tensorflow as tf
from tensorflow.keras import backend as K
import numpy as np
import time
#from Models.loss import smoothL1
from tensorflow.keras.models import Model
from tensorflow.keras.models import load_model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Dense, Dropout, Input, InputLayer
from tensorflow.keras.layers import Activation, Flatten, BatchNormalization
from tensorflow.keras.regularizers import l2

from tf_keras_vis.gradcam import Gradcam
from tf_keras_vis.gradcam import GradcamPlusPlus

from tf_keras_vis.utils import normalize

import matplotlib.pyplot as plt



import math
import io
import os
from collections import namedtuple
import sys

import operator

import copy

#from tf_keras_vis.utils import num_of_gpus

#_, gpus = num_of_gpus()
#print('{} GPUs'.format(gpus))
!pwd
os.chdir("./drive/MyDrive/Colab Notebooks")
!pwd
!ls

print(tf.__version__)

from platform import python_version

print(python_version())

     |████████████████████████████████| 53 kB 1.3 MB/s 
/content
/content/drive/MyDrive/Colab Notebooks
 0_0.npy
 10_N_MNIST_GEN_SET_25_2.txt
 10_N_MNIST_GEN_SET_LABELS_25_2.txt
 25_N_CIFAR_GEN_SET_50_10.txt
 25_N_CIFAR_GEN_SET_LABELS_50_10.txt
'ACASX keras model.ipynb'
 ACASX_Prophecy.ipynb
'Anchor on tabular data.ipynb'
'Anchor SCENIC.ipynb'
'Backdoor DFST.ipynb'
 backdoor_test_X_cifar.txt
 backdoor_test_X_GTSRB.txt
 backdoor_test_X_MNIST.txt
'BoeingTaxiNet (1).ipynb'
 BoeingTaxiNet.ipynb
'Boeing TaxiNet Prophecy Runtime_v2.ipynb'
'Boeing TaxiNet Visualization (1).ipynb'
'Boeing TaxiNet Visualization.ipynb'
 BOEING_YOLO.ipynb
 cifar10b_sunrise.h5
 cifar10_normalization.h5
 cifar10-poisoned.hdf5
 cifar10_sunrise.h5
 CIFAR_GEN_SET_50_10.txt
 CIFAR_GEN_SET_LABELS_50_10.txt
 cifar.hdf5
 CIFAR-Original.ipynb
 cifar_test
 CIFAR_VAL_SET_50_10.txt
 CIFAR_VAL_SET_LABELS_50_10.txt
'Compare DNN-Coverage.ipynb'
'ConcolicNN 2.ipynb'
 config.py.ipynb
'ConstructTestSet1 (1).py'
'ConstructTestSet1 (

In [ ]:
import os
import keras
import argparse
import numpy as np
import tensorflow as tf
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
#from keras.layers.normalization import BatchNormalization
from keras.layers import Conv2D, Dense, Input, add, Activation, GlobalAveragePooling2D
from keras.callbacks import LearningRateScheduler
from keras.models import Model, load_model
from keras import optimizers, regularizers
from keras import backend as K
import pickle

def get_prediction(img, tensor = None):

    if tensor == "ALL":
        full_list = []
        for tensor in ['dense_1', 'dense_2', 'dense_3']:
            layer_name = tensor
            layer = model.get_layer(layer_name)
            func = K.function(model.input, [layer.output])
            imag = np.expand_dims(img,axis=0)
            outputs = func(imag)
           # print(outputs)
            layer_outputs_test = []
            fingerprint1 = ((outputs[0] > 0.0).astype('int'))
            layer_outputs_test = []
            layer_result = fingerprint1
            layer_outputs_test.append(layer_result[0])
            full_list.append(layer_outputs_test)

        new_list = [list(full_list[0][0])+list(full_list[1][0])+list(full_list[2][0])]
        layer_outputs_test = np.asarray(new_list)
        return layer_outputs_test
    else:
        layer_name = tensor
        layer = model.get_layer(layer_name)
        func = K.function(model.input, [layer.output])
        imag = np.expand_dims(img,axis=0)
        outputs = func(imag)
       # print(outputs)
        fingerprint1 = ((outputs[0] > 0.0).astype('int'))
        layer_outputs_test = []
        layer_result = fingerprint1
        layer_outputs_test.append(layer_result[0])
        layer_outputs_test = np.asarray(layer_outputs_test)

        return layer_outputs_test

def get_prediction_vals(img, tensor = None):

    if tensor == "ALL":
        full_list = []
        for tensor in ['activation_1','activation_2','activation_3']:
            layer_name = tensor
            layer = model.get_layer(layer_name)
            func = K.function(model.input, [layer.output])
            imag = np.expand_dims(img,axis=0)
            outputs = func(imag)
            op = np.array(outputs)
            #print(op.shape)
            flat_op = op.flatten()
            #print(flat_op.shape)
            fingerprint1 = list(flat_op)
            #print(layer.name,fingerprint1)
            layer_outputs_test = []
            layer_result = fingerprint1
            layer_outputs_test.append(layer_result)
            full_list.append(layer_outputs_test)

        new_list = [list(full_list[0][0])+list(full_list[1][0])+list(full_list[2][0])]
        layer_outputs_test = np.asarray(new_list)
        return layer_outputs_test
    else:
        layer_name = tensor
        layer = model.get_layer(layer_name)
        func = K.function(model.input, [layer.output])
        imag = np.expand_dims(img,axis=0)
        outputs = func(imag)
       # print(outputs)
        #fingerprint1 = ((outputs[0] > 0.0).astype('int'))
        fingerprint1 = outputs[0]
        layer_outputs_test = []
        layer_result = fingerprint1
        layer_outputs_test.append(layer_result[0])
        layer_outputs_test = np.asarray(layer_outputs_test)

        return layer_outputs_test





def fingerprint_suffix(img,ten):
    return (get_prediction(img, tensor=ten)>0.0).astype('int')

def fingerprint_suffix_vals(img,ten):
    return get_prediction_vals(img, tensor=ten)

#from tf_keras_vis.gradcam import Gradcam
#from tf_keras_vis.gradcam import GradcamPlusPlus

def model_modifier_act1_layer(current_model):
    target_layer = current_model.get_layer(name='activation_1') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_act2_layer(current_model):
    target_layer = current_model.get_layer(name='activation_2') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_act3_layer(current_model):
    target_layer = current_model.get_layer(name='activation_3') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_last_layer(current_model):
    target_layer = current_model.get_layer(name='activation_5') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_layer(current_model):
    target_layer = current_model.get_layer(name='dense') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=current_model.inputs,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def model_modifier_layer_inps(current_model):
    src_layer = current_model.get_layer(name='activation')
    target_layer = current_model.get_layer(name='dense') #layer is the name of the intermediate layer
    new_model = tf.keras.Model(inputs=src_layer.input,
                               outputs=target_layer.output)
    new_model.layers[-1].activation = tf.keras.activations.linear
    return new_model

def loss_gen_sum(node_list):
    def loss(output):
        print(output.shape)
        op = np.array(output)

        #if (op.ndim > 2):
        op = op.flatten()
        output = np.empty((1,len(op)))
        print(output.shape)
        output[0] = op

        loss_val = sum([output[0][i] for i in node_list])/len(node_list)
        return loss_val
    return loss


def loss_gen_sum1(node_list):
    def loss(output):

        loss_val = sum([output[0][i] for i in node_list])/len(node_list)
        return loss_val
    return loss

def loss_gen_sep(node):
    def loss(output):
        loss_val = output[0][node]
        return loss_val
    return loss

#import matplotlib.pyplot as plt
#from keras.models import load_model
print('Loading the model:')
model=load_model('mnist0_poisoned.h5')#,custom_objects={'smoothL1':smoothL1})
print("Printing summary of the model:")
model.summary()


layer_name = 'flatten_1'
lay = model.get_layer(layer_name)
print(lay.get_config())

layer_name = 'dense_1'
lay = model.get_layer(layer_name)
print(lay.get_config())

layer_name = 'activation_3'
lay = model.get_layer(layer_name)
print(lay.get_config())

layer_name = 'dense_2'
lay = model.get_layer(layer_name)
print(lay.get_config())






Loading the model:
Printing summary of the model:
Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_1 (Conv2D)           (None, 26, 26, 2)         20        
                                                                 
 activation_1 (Activation)   (None, 26, 26, 2)         0         
                                                                 
 conv2d_2 (Conv2D)           (None, 24, 24, 4)         76        
                                                                 
 activation_2 (Activation)   (None, 24, 24, 4)         0         
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 12, 12, 4)        0         
 2D)                                                             
                                                                 
 flatten_1 (Flatten)         (None, 576)               0         
    

In [ ]:
def check_pattern(layer_vals,suff,neuron_ids,neuron_sig,VAL = False,ALL = False):
   #layer = 'activation_3'  #layer name, get it from the model summary
   #neuron_ids =  [-1,71,-1,20,-1,1,-1,28,-1,79,-1,29,-1,39,-1,49,-1,42,-1,33,-1,114,-1,117,-1,88,-1,31,-1,36,-1,33,-1,17,-1,117,-1,1,-1,28,-1,97] # neuron ids in the pattern
   #neuron_sig =  ['>',0.435630053,'>',4.067234278,'<=',42.78802872,'<=',20.19964218,'<=',41.11421585,'<=',23.87646389,'<=',15.27161694,'<=',36.27468491,'<=',30.84732342,'<=',40.63782883,'<=',25.80421829,'<=',43.16035843,'<=',38.44324303,'<=',66.91875458,'<=',26.18983078,'<=',16.70914364,'<=',41.36343193,'<=',35.72474098,'<=',35.5690403,'<=',14.66370487,'<=',40.59473419189453]

   if (VAL == False):
      if ((suff[:,neuron_ids][0] == neuron_sig).all(axis=0)):
        return True
      else:
        return False

   found = True;

   oper = -1
   for ind in range(0,len(neuron_ids)):
     if (ind % 2 == 0):
       op = neuron_sig[ind]
       if (op == '<='):
         oper = 0
       else:
         oper = 1
     else:
       v = int(neuron_ids[ind])
       vsig = float(neuron_sig[ind])
       val = layer_vals[v]
       #print(v,vsig,val,oper)
       if (oper == 0):
         if (val > vsig):
           found = False
           break
       else:
         if (val <= vsig):
            found = False
            break
       oper = -1

   return found

In [ ]:
import numpy as np
import csv

num_classes = 10

a=[]
path = 'test_Y_MNIST.txt'
inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
for row in inputfile: # TRAVERSES ALL ROWS OF DATA
      c=[]
      for i in range(0,len(row)):
          c.append(int(row[i]))
      a.append(c[0])
testFULL_Y = np.array(a)
print("FULL IDEAL LABELS:", testFULL_Y.shape)


a=[]
path = 'test_X_MNIST.txt'
inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
for row in inputfile: # TRAVERSES ALL ROWS OF DATA
    c=[]
    for i in range(0,len(row)):
        c.append(int(row[i]))
    c = np.reshape(c, (28,28,1))
    a.append(c)
testFULL_X = np.array(a)
print("FULL CLEAN DATA:", testFULL_X.shape)



test_labels = (model.predict(testFULL_X)).argmax(axis=1)
print("FULL TEST LABELS:", test_labels.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels)):
  if (test_labels[indx] == testFULL_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)


### RANDOM SPLIT INTO VAL AND TEST SET
import random
length = (len(testFULL_X))
ind = 0
valinds = []
while (ind < (length)/2):
  ix = random.randint(0,length)
  while(ix in valinds):
    ix = random.randint(0,length)
  valinds.append(ix)
  ind = ind + 1

test_X = []
test_Y = []
val_X = []
val_Y = []
for indx in range(0,length):
  if (indx in valinds):
    val_X.append(testFULL_X[indx])
    val_Y.append(testFULL_Y[indx])
  else:
    test_X.append(testFULL_X[indx])
    test_Y.append(testFULL_Y[indx])

val_X = np.array(val_X)
val_Y = np.array(val_Y)
test_X = np.array(test_X)
test_Y = np.array(test_Y)
print("VALIDATION SET:", len(val_X), val_X.shape, len(val_Y), val_Y.shape)
print("TEST SET:", len(test_X), test_X.shape, len(test_Y), test_Y.shape)

val_labels = (model.predict(val_X)).argmax(axis=1)
print("ACTUAL Val LABELS:", val_labels.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(val_labels)):
  if (val_labels[indx] == val_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",VAL ACCURACY:", (PASS/(PASS+FAIL))*100.0)

test_labels = (model.predict(test_X)).argmax(axis=1)
print("ACTUAL TEST LABELS:", test_labels.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels)):
  if (test_labels[indx] == test_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)

gen_data = np.array(test_X)
gen_labels = np.array(test_Y)
gen_act_labels = (model.predict(gen_data)).argmax(axis=1)
print("ACTUAL GEN LABELS:", gen_act_labels.shape)

label0_tots = len(np.where(gen_act_labels == 0)[0])
label1_tots = len(np.where(gen_act_labels == 1)[0])
label2_tots = len(np.where(gen_act_labels == 2)[0])
label3_tots = len(np.where(gen_act_labels == 3)[0])
label4_tots = len(np.where(gen_act_labels == 4)[0])
label5_tots = len(np.where(gen_act_labels == 5)[0])
label6_tots = len(np.where(gen_act_labels == 6)[0])
label7_tots = len(np.where(gen_act_labels == 7)[0])
label8_tots = len(np.where(gen_act_labels == 8)[0])
label9_tots = len(np.where(gen_act_labels == 9)[0])

print(label0_tots,label1_tots,label2_tots,label3_tots,label4_tots,label5_tots)
print(label6_tots,label7_tots,label8_tots,label9_tots)



Pgen_data = []
Pgen_labels = []
PASS = 0
FAIL = 0
for indx in range(0, len(gen_labels)):
  Pgen_data.append(gen_data[indx])
  Pgen_labels.append(gen_labels[indx])
  if (gen_act_labels[indx] == gen_labels[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",GEN ACCURACY:", (PASS/(PASS+FAIL))*100.0)





a=[]
path = 'backdoor_test_X_MNIST.txt'
ind = 0
inputfile = csv.reader(open(path,'r')) # DEFINES THE INPUT FILE
for row in inputfile: # TRAVERSES ALL ROWS OF DATA
    c=[]
    for i in range(0,len(row)):
     #   c.append(float(row[i])/255.0)
        c.append(int(row[i]))
    c = np.reshape(c, (28,28,1))
    a.append(c)
    ind = ind + 1

poisoned_x = np.array(a)
print("POISONED DATA:", poisoned_x.shape)


test_labels_pois = (model.predict(poisoned_x)).argmax(axis=1)
print("POIS TEST LABELS:", test_labels_pois.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(test_labels_pois)):
  if (test_labels_pois[indx] == testFULL_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",TEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)

### RANDOM POISON DATA SPLIT INTO VAL AND TEST SET
import random
val_length = int((len(poisoned_x)) * (1/2))
ind = 0
valinds = []
while (ind < val_length):
  ix = random.randint(0,val_length)
  while(ix in valinds):
    ix = random.randint(0,val_length)
  valinds.append(ix)
  ind = ind + 1


Pval_X = []
Pval_Y = []
PTEST_X = []
PTEST_Y = []
for indx in range(0,len(poisoned_x)):
  if (indx in valinds):
    Pval_X.append(poisoned_x[indx])
    Pval_Y.append(testFULL_Y[indx])
  else:
    PTEST_X.append(poisoned_x[indx])
    PTEST_Y.append(testFULL_Y[indx])

Pval_X = np.array(Pval_X)
Pval_Y = np.array(Pval_Y)
print("VALIDATION SET:", len(Pval_X), Pval_X.shape, len(Pval_Y), Pval_Y.shape)


Pval_labels = (model.predict(Pval_X)).argmax(axis=1)
print("ACTUAL Val LABELS:", Pval_labels.shape)
PASS = 0
FAIL = 0
for indx in range(0, len(Pval_labels)):
  if (Pval_labels[indx] == Pval_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",PVAL ACCURACY:", (PASS/(PASS+FAIL))*100.0)

################################################################################

SPLIT = (25.0)/(100.0)
import random
test_length = int((len(PTEST_X)) * SPLIT)
ind = 0
testinds = []
while (ind < test_length):
  ix = random.randint(0,test_length)
  while(ix in testinds):
    ix = random.randint(0,test_length)
  testinds.append(ix)
  ind = ind + 1

Ptest_X = []
Ptest_Y = []
for indx in range(0,len(PTEST_X)):
  if (indx in testinds):
    Ptest_X.append(PTEST_X[indx])
    Ptest_Y.append(PTEST_Y[indx])


Ptest_X = np.array(Ptest_X)
Ptest_Y = np.array(Ptest_Y)
print("TEST SET:", len(Ptest_X), Ptest_X.shape, len(Ptest_Y), Ptest_Y.shape)


Ptest_labels = (model.predict(Ptest_X)).argmax(axis=1)
print("ACTUAL TEST LABELS:", Ptest_labels.shape)

PASS = 0
FAIL = 0
for indx in range(0, len(Ptest_labels)):
  Pgen_data.append(Ptest_X[indx])
  Pgen_labels.append(Ptest_Y[indx])
  if (Ptest_labels[indx] == Ptest_Y[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",PTEST ACCURACY:", (PASS/(PASS+FAIL))*100.0)

Pgen_data = np.array(Pgen_data)
Pgen_labels = np.array(Pgen_labels)
print("Pgen", Pgen_data.shape, Pgen_labels.shape )
Pgen_act_labels = (model.predict(Pgen_data)).argmax(axis=1)
print("ACTUAL GEN LABELS:", Pgen_act_labels.shape)

PASS = 0
FAIL = 0
for indx in range(0, len(Pgen_labels)):
  if (Pgen_act_labels[indx] == Pgen_labels[indx]):
    PASS = PASS + 1
  else:
    FAIL = FAIL + 1
print("PASS:", PASS, ",FAIL:", FAIL, ",PGEN ACCURACY:", (PASS/(PASS+FAIL))*100.0)





FULL IDEAL LABELS: (10000,)
FULL CLEAN DATA: (10000, 28, 28, 1)
FULL TEST LABELS: (10000,)
PASS: 9863 ,FAIL: 137 ,TEST ACCURACY: 98.63
VALIDATION SET: 5000 (5000, 28, 28, 1) 5000 (5000,)
TEST SET: 5000 (5000, 28, 28, 1) 5000 (5000,)
ACTUAL Val LABELS: (5000,)
PASS: 4934 ,FAIL: 66 ,VAL ACCURACY: 98.68
ACTUAL TEST LABELS: (5000,)
PASS: 4929 ,FAIL: 71 ,TEST ACCURACY: 98.58
ACTUAL GEN LABELS: (5000,)
493 595 520 507 496 417
493 481 485 513
PASS: 4929 ,FAIL: 71 ,GEN ACCURACY: 98.58
POISONED DATA: (10000, 28, 28, 1)
POIS TEST LABELS: (10000,)
PASS: 1038 ,FAIL: 8962 ,TEST ACCURACY: 10.38
VALIDATION SET: 5000 (5000, 28, 28, 1) 5000 (5000,)
ACTUAL Val LABELS: (5000,)
PASS: 520 ,FAIL: 4480 ,PVAL ACCURACY: 10.4
TEST SET: 1250 (1250, 28, 28, 1) 1250 (1250,)
ACTUAL TEST LABELS: (1250,)
PASS: 123 ,FAIL: 1127 ,PTEST ACCURACY: 9.84
Pgen (6250, 28, 28, 1) (6250,)
ACTUAL GEN LABELS: (6250,)
PASS: 5052 ,FAIL: 1198 ,PGEN ACCURACY: 80.83200000000001


In [ ]:
#COLLECTING DATA FOR DECISION TREE LEARNING
import time
extract_pat_start_time = time.time()

layer = model.get_layer('dense_1') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(gen_data)
fingerprint_1 = ((outputs[0] > 0.0).astype('int'))
fingerprint1 = outputs[0]

layer = model.get_layer('activation_3') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(gen_data)
fingerprint_2 = ((outputs[0] > 0.0).astype('int'))
fingerprint2 = outputs[0]

layer = model.get_layer('dense_2') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(gen_data)
fingerprint_3 = ((outputs[0] > 0.0).astype('int'))
fingerprint3 = outputs[0]



PASS = 0
FAIL = 0
val_vec1 = []
val_vec_labels1 = []
actuals_gen = (model.predict(gen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(fingerprint1)):
  #print(actuals_gen[i],gen_labels[i])
  if (actuals_gen[i] == gen_labels[i]):
    PASS = PASS + 1
    val_vec_labels1.append(gen_labels[i])
    val_vec1.append(np.asarray((fingerprint1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    val_vec_labels1.append(10 + actuals_gen[i])
    val_vec1.append(np.asarray((fingerprint1[i]).flatten()))

PASS = 0
FAIL = 0
act_vec1 = []
act_vec_labels1 = []
print("ACT from layer:")
for i in range(0,len(fingerprint_1)):
  if (actuals_gen[i] == gen_labels[i]):
    PASS = PASS + 1
    act_vec_labels1.append(gen_labels[i])
    act_vec1.append(np.asarray((fingerprint_1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    act_vec_labels1.append(10 + actuals_gen[i])
    act_vec1.append(np.asarray((fingerprint_1[i]).flatten()))

PASS = 0
FAIL = 0
val_vec2 = []
val_vec_labels2 = []
actuals_gen = (model.predict(gen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(fingerprint2)):
  if (actuals_gen[i] == gen_labels[i]):
    PASS = PASS + 1
    val_vec_labels2.append(gen_labels[i])
    val_vec2.append(np.asarray((fingerprint2[i]).flatten()))
  else:
    FAIL = FAIL + 1
    val_vec_labels2.append(10 + actuals_gen[i])
    val_vec2.append(np.asarray((fingerprint2[i]).flatten()))

PASS = 0
FAIL = 0
act_vec2 = []
act_vec_labels2 = []
print("ACT from layer:")
for i in range(0,len(fingerprint_2)):
  if (actuals_gen[i] == gen_labels[i]):
    PASS = PASS + 1
    act_vec_labels2.append(gen_labels[i])
    act_vec2.append(np.asarray((fingerprint_2[i]).flatten()))
  else:
    FAIL = FAIL + 1
    act_vec_labels2.append(10 + actuals_gen[i])
    act_vec2.append(np.asarray((fingerprint_2[i]).flatten()))

PASS = 0
FAIL = 0
val_vec3 = []
val_vec_labels3 = []
actuals_gen = (model.predict(gen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(fingerprint3)):
  if (actuals_gen[i] == gen_labels[i]):
    PASS = PASS + 1
    val_vec_labels3.append(gen_labels[i])
    val_vec3.append(np.asarray((fingerprint3[i]).flatten()))
  else:
    FAIL = FAIL + 1
    val_vec_labels3.append(10 + actuals_gen[i])
    val_vec3.append(np.asarray((fingerprint3[i]).flatten()))

PASS = 0
FAIL = 0
act_vec3 = []
act_vec_labels3 = []
print("ACT from layer:")
for i in range(0,len(fingerprint_3)):
  if (actuals_gen[i] == gen_labels[i]):
    PASS = PASS + 1
    act_vec_labels3.append(gen_labels[i])
    act_vec3.append(np.asarray((fingerprint_3[i]).flatten()))
  else:
    FAIL = FAIL + 1
    act_vec_labels3.append(10 + actuals_gen[i])
    act_vec3.append(np.asarray((fingerprint_3[i]).flatten()))


VALS from layer:
ACT from layer:
VALS from layer:
ACT from layer:
VALS from layer:
ACT from layer:


In [ ]:
#layer_preds
layer = model.get_layer('dense_1') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(Pgen_data)
Pfingerprint_1 = ((outputs[0] > 0.0).astype('int'))
Pfingerprint1 = outputs[0]

layer = model.get_layer('activation_3') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(Pgen_data)
Pfingerprint_2 = ((outputs[0] > 0.0).astype('int'))
Pfingerprint2 = outputs[0]

layer = model.get_layer('dense_2') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(Pgen_data)
Pfingerprint_3 = ((outputs[0] > 0.0).astype('int'))
Pfingerprint3 = outputs[0]


PASS = 0
FAIL = 0
FAIL_P = 0
Pval_vec1 = []
Pval_vec_labels1 = []
actuals_gen = (model.predict(Pgen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(Pfingerprint1)):
  #print(actuals_gen[i],gen_labels[i])
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Pval_vec_labels1.append(Pgen_labels[i])
    Pval_vec1.append(np.asarray((Pfingerprint1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 7):
      FAIL_P = FAIL_P + 1
      Pval_vec_labels1.append(1000)
    else:
      Pval_vec_labels1.append(10 + actuals_gen[i])
    Pval_vec1.append(np.asarray((Pfingerprint1[i]).flatten()))

PASS = 0
FAIL = 0
Pact_vec1 = []
Pact_vec_labels1 = []
print("ACT from layer:")
for i in range(0,len(Pfingerprint_1)):
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Pact_vec_labels1.append(Pgen_labels[i])
    Pact_vec1.append(np.asarray((Pfingerprint_1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 7):
      Pact_vec_labels1.append(1000)
    else:
      Pact_vec_labels1.append(10 + actuals_gen[i])
    Pact_vec1.append(np.asarray((Pfingerprint_1[i]).flatten()))

PASS = 0
FAIL = 0
Pval_vec2 = []
Pval_vec_labels2 = []
actuals_gen = (model.predict(Pgen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(Pfingerprint2)):
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Pval_vec_labels2.append(Pgen_labels[i])
    Pval_vec2.append(np.asarray((Pfingerprint2[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 7):
      Pval_vec_labels2.append(1000)
    else:
      Pval_vec_labels2.append(10 + actuals_gen[i])
    Pval_vec2.append(np.asarray((Pfingerprint2[i]).flatten()))

PASS = 0
FAIL = 0
Pact_vec2 = []
Pact_vec_labels2 = []
print("ACT from layer:")
for i in range(0,len(Pfingerprint_2)):
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Pact_vec_labels2.append(Pgen_labels[i])
    Pact_vec2.append(np.asarray((Pfingerprint_2[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 7):
      Pact_vec_labels2.append(1000)
    else:
      Pact_vec_labels2.append(10 + actuals_gen[i])
    Pact_vec2.append(np.asarray((Pfingerprint_2[i]).flatten()))

PASS = 0
FAIL = 0
Pval_vec3 = []
Pval_vec_labels3 = []
actuals_gen = (model.predict(Pgen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(Pfingerprint3)):
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Pval_vec_labels3.append(Pgen_labels[i])
    Pval_vec3.append(np.asarray((Pfingerprint3[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 7):
      Pval_vec_labels3.append(1000)
    else:
      Pval_vec_labels3.append(10 + actuals_gen[i])
    Pval_vec3.append(np.asarray((Pfingerprint3[i]).flatten()))

PASS = 0
FAIL = 0
Pact_vec3 = []
Pact_vec_labels3 = []
print("ACT from layer:")
for i in range(0,len(Pfingerprint_3)):
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Pact_vec_labels3.append(Pgen_labels[i])
    Pact_vec3.append(np.asarray((Pfingerprint_3[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 7):
      Pact_vec_labels3.append(1000)
    else:
      Pact_vec_labels3.append(10 + actuals_gen[i])
    Pact_vec3.append(np.asarray((Pfingerprint_3[i]).flatten()))


print("PASS:", PASS, ", FAIL:", FAIL, ",ACC:", ((PASS/(PASS+FAIL)) * 100.0))
print(", FAIL POISONED:", FAIL_P)


VALS from layer:
ACT from layer:
VALS from layer:
ACT from layer:
VALS from layer:
ACT from layer:
PASS: 5052 , FAIL: 1198 ,ACC: 80.83200000000001
, FAIL POISONED: 1134


In [ ]:
#layer_preds
layer = model.get_layer('dense_1') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(Pgen_data)
Rfingerprint_1 = ((outputs[0] > 0.0).astype('int'))
Rfingerprint1 = outputs[0]

layer = model.get_layer('activation_3') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(Pgen_data)
Rfingerprint_2 = ((outputs[0] > 0.0).astype('int'))
Rfingerprint2 = outputs[0]

layer = model.get_layer('dense_2') # activation, activation_2, activation_3, activation_4, activation_5
func = K.function(model.input, [layer.output])
outputs = func(Pgen_data)
Rfingerprint_3 = ((outputs[0] > 0.0).astype('int'))
Rfingerprint3 = outputs[0]





PASS = 0
FAIL = 0
FAIL_P = 0
Rval_vec1 = []
Rval_vec_labels1 = []
actuals_gen = (model.predict(Pgen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(Rfingerprint1)):
  #print(actuals_gen[i],gen_labels[i])
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Rval_vec_labels1.append(Pgen_labels[i])
    Rval_vec1.append(np.asarray((Rfingerprint1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 7):
      FAIL_P = FAIL_P + 1
      Rval_vec_labels1.append(Pgen_labels[i])
    else:
      Rval_vec_labels1.append(10 + actuals_gen[i])
    Rval_vec1.append(np.asarray((Rfingerprint1[i]).flatten()))

PASS = 0
FAIL = 0
Ract_vec1 = []
Ract_vec_labels1 = []
print("ACT from layer:")
for i in range(0,len(Rfingerprint_1)):
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Ract_vec_labels1.append(Pgen_labels[i])
    Ract_vec1.append(np.asarray((Rfingerprint_1[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 7):
      Ract_vec_labels1.append(Pgen_labels[i])
    else:
      Ract_vec_labels1.append(10 + actuals_gen[i])
    Ract_vec1.append(np.asarray((Rfingerprint_1[i]).flatten()))

PASS = 0
FAIL = 0
Rval_vec2 = []
Rval_vec_labels2 = []
actuals_gen = (model.predict(Pgen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(Rfingerprint2)):
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Rval_vec_labels2.append(Pgen_labels[i])
    Rval_vec2.append(np.asarray((Rfingerprint2[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 7):
      Rval_vec_labels2.append(Pgen_labels[i])
    else:
      Rval_vec_labels2.append(10 + actuals_gen[i])
    Rval_vec2.append(np.asarray((Rfingerprint2[i]).flatten()))

PASS = 0
FAIL = 0
Ract_vec2 = []
Ract_vec_labels2 = []
print("ACT from layer:")
for i in range(0,len(Rfingerprint_2)):
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Ract_vec_labels2.append(Pgen_labels[i])
    Ract_vec2.append(np.asarray((Rfingerprint_2[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 7):
      Ract_vec_labels2.append(Pgen_labels[i])
    else:
      Ract_vec_labels2.append(10 + actuals_gen[i])
    Ract_vec2.append(np.asarray((Rfingerprint_2[i]).flatten()))


PASS = 0
FAIL = 0
Rval_vec3 = []
Rval_vec_labels3 = []
actuals_gen = (model.predict(Pgen_data)).argmax(axis=1)
print("VALS from layer:")
for i in range(0,len(Rfingerprint3)):
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Rval_vec_labels3.append(Pgen_labels[i])
    Rval_vec3.append(np.asarray((Rfingerprint3[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 7):
      Rval_vec_labels3.append(Pgen_labels[i])
    else:
      Rval_vec_labels3.append(10 + actuals_gen[i])
    Rval_vec3.append(np.asarray((Rfingerprint3[i]).flatten()))

PASS = 0
FAIL = 0
Ract_vec3 = []
Ract_vec_labels3 = []
print("ACT from layer:")
for i in range(0,len(Rfingerprint_3)):
  if (actuals_gen[i] == Pgen_labels[i]):
    PASS = PASS + 1
    Ract_vec_labels3.append(Pgen_labels[i])
    Ract_vec3.append(np.asarray((Rfingerprint_3[i]).flatten()))
  else:
    FAIL = FAIL + 1
    if (actuals_gen[i] == 7):
      Ract_vec_labels3.append(Pgen_labels[i])
    else:
      Ract_vec_labels3.append(10 + actuals_gen[i])
    Ract_vec3.append(np.asarray((Rfingerprint_3[i]).flatten()))


print("PASS:", PASS, ", FAIL:", FAIL, ",ACC:", ((PASS/(PASS+FAIL)) * 100.0))
print(", FAIL POISONED:", FAIL_P)


VALS from layer:
ACT from layer:
VALS from layer:
ACT from layer:
VALS from layer:
ACT from layer:
PASS: 5052 , FAIL: 1198 ,ACC: 80.83200000000001
, FAIL POISONED: 1134


In [ ]:
from sklearn import tree

val_vec1 = np.asarray(val_vec1)
val_vec_labels1 = np.asarray(val_vec_labels1)
print(val_vec1.shape, val_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
basic_estimator1 = tree.DecisionTreeClassifier()
basic_estimator1.fit(val_vec1, val_vec_labels1)

act_vec1 = np.asarray(act_vec1)
act_vec_labels1 = np.asarray(act_vec_labels1)
print(act_vec1.shape, act_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
basic_estimator1a = tree.DecisionTreeClassifier()
basic_estimator1a.fit(act_vec1, act_vec_labels1)

val_vec2 = np.asarray(val_vec2)
val_vec_labels2 = np.asarray(val_vec_labels2)
print(val_vec2.shape, val_vec_labels2.shape)
basic_estimator2 = tree.DecisionTreeClassifier()
basic_estimator2.fit(val_vec2, val_vec_labels2)

act_vec2 = np.asarray(act_vec2)
act_vec_labels2 = np.asarray(act_vec_labels2)
print(act_vec2.shape, act_vec_labels2.shape)
## PATTERNs WITH ACTUAL LABELS
basic_estimator2a = tree.DecisionTreeClassifier()
basic_estimator2a.fit(act_vec2, act_vec_labels2)

val_vec3 = np.asarray(val_vec3)
val_vec_labels3 = np.asarray(val_vec_labels3)
print(val_vec3.shape, val_vec_labels3.shape)
basic_estimator3 = tree.DecisionTreeClassifier()
basic_estimator3.fit(val_vec3, val_vec_labels3)

act_vec3 = np.asarray(act_vec3)
act_vec_labels3 = np.asarray(act_vec_labels3)
print(act_vec3.shape, act_vec_labels3.shape)
## PATTERNs WITH ACTUAL LABELS
basic_estimator3a = tree.DecisionTreeClassifier()
basic_estimator3a.fit(act_vec3, act_vec_labels3)

#extract_pat_time = time.time() - extract_pat_start_time

#print("Patterns Extraction Time:", extract_pat_time)


(5000, 128) (5000,)
(5000, 128) (5000,)
(5000, 128) (5000,)
(5000, 128) (5000,)
(5000, 10) (5000,)
(5000, 10) (5000,)


DecisionTreeClassifier()

In [ ]:
from sklearn import tree

Pval_vec1 = np.asarray(Pval_vec1)
Pval_vec_labels1 = np.asarray(Pval_vec_labels1)
print(Pval_vec1.shape, Pval_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
Pbasic_estimator1 = tree.DecisionTreeClassifier()
Pbasic_estimator1.fit(Pval_vec1, Pval_vec_labels1)

Pact_vec1 = np.asarray(Pact_vec1)
Pact_vec_labels1 = np.asarray(Pact_vec_labels1)
print(Pact_vec1.shape, Pact_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
Pbasic_estimator1a = tree.DecisionTreeClassifier()
Pbasic_estimator1a.fit(Pact_vec1, Pact_vec_labels1)

Pval_vec2 = np.asarray(Pval_vec2)
Pval_vec_labels2 = np.asarray(Pval_vec_labels2)
print(Pval_vec2.shape, Pval_vec_labels2.shape)
Pbasic_estimator2 = tree.DecisionTreeClassifier()
Pbasic_estimator2.fit(Pval_vec2, Pval_vec_labels2)

Pact_vec2 = np.asarray(Pact_vec2)
Pact_vec_labels2 = np.asarray(Pact_vec_labels2)
print(Pact_vec2.shape, Pact_vec_labels2.shape)
## PATTERNs WITH ACTUAL LABELS
Pbasic_estimator2a = tree.DecisionTreeClassifier()
Pbasic_estimator2a.fit(Pact_vec2, Pact_vec_labels2)

Pval_vec3 = np.asarray(Pval_vec3)
Pval_vec_labels3 = np.asarray(Pval_vec_labels3)
print(Pval_vec3.shape, Pval_vec_labels3.shape)
Pbasic_estimator3 = tree.DecisionTreeClassifier()
Pbasic_estimator3.fit(Pval_vec3, Pval_vec_labels3)

Pact_vec3 = np.asarray(Pact_vec3)
Pact_vec_labels3 = np.asarray(Pact_vec_labels3)
print(Pact_vec3.shape, Pact_vec_labels3.shape)
## PATTERNs WITH ACTUAL LABELS
Pbasic_estimator3a = tree.DecisionTreeClassifier()
Pbasic_estimator3a.fit(Pact_vec3, Pact_vec_labels3)




(6250, 128) (6250,)
(6250, 128) (6250,)
(6250, 128) (6250,)
(6250, 128) (6250,)
(6250, 10) (6250,)
(6250, 10) (6250,)


DecisionTreeClassifier()

In [ ]:
from sklearn import tree

Rval_vec1 = np.asarray(Rval_vec1)
Rval_vec_labels1 = np.asarray(Rval_vec_labels1)
print(Rval_vec1.shape, Rval_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
Rbasic_estimator1 = tree.DecisionTreeClassifier()
Rbasic_estimator1.fit(Rval_vec1, Rval_vec_labels1)

Ract_vec1 = np.asarray(Ract_vec1)
Ract_vec_labels1 = np.asarray(Ract_vec_labels1)
print(Ract_vec1.shape, Ract_vec_labels1.shape)
## PATTERNs WITH ACTUAL LABELS
Rbasic_estimator1a = tree.DecisionTreeClassifier()
Rbasic_estimator1a.fit(Ract_vec1, Ract_vec_labels1)

Rval_vec2 = np.asarray(Rval_vec2)
Rval_vec_labels2 = np.asarray(Rval_vec_labels2)
print(Rval_vec2.shape, Rval_vec_labels2.shape)
Rbasic_estimator2 = tree.DecisionTreeClassifier()
Rbasic_estimator2.fit(Rval_vec2, Rval_vec_labels2)

Ract_vec2 = np.asarray(Ract_vec2)
Ract_vec_labels2 = np.asarray(Ract_vec_labels2)
print(Ract_vec2.shape, Ract_vec_labels2.shape)
## PATTERNs WITH ACTUAL LABELS
Rbasic_estimator2a = tree.DecisionTreeClassifier()
Rbasic_estimator2a.fit(Ract_vec2, Ract_vec_labels2)

Rval_vec3 = np.asarray(Rval_vec3)
Rval_vec_labels3 = np.asarray(Rval_vec_labels3)
print(Rval_vec3.shape, Rval_vec_labels3.shape)
Rbasic_estimator3 = tree.DecisionTreeClassifier()
Rbasic_estimator3.fit(Rval_vec3, Rval_vec_labels3)

Ract_vec3 = np.asarray(Ract_vec3)
Ract_vec_labels3 = np.asarray(Ract_vec_labels3)
print(Ract_vec3.shape, Ract_vec_labels3.shape)
## PATTERNs WITH ACTUAL LABELS
Rbasic_estimator3a = tree.DecisionTreeClassifier()
Rbasic_estimator3a.fit(Ract_vec3, Ract_vec_labels3)

extract_pat_time = time.time() - extract_pat_start_time

print("Patterns Extraction Time:", extract_pat_time)

(6250, 128) (6250,)
(6250, 128) (6250,)
(6250, 128) (6250,)
(6250, 128) (6250,)
(6250, 10) (6250,)
(6250, 10) (6250,)
Patterns Extraction Time: 39.39788746833801


In [ ]:
from tqdm import tqdm
import operator
import pandas as pd

train_suffixes = act_vec1
train_labels = act_vec_labels1

def get_decision_path(estimator, inp):
  # Extract the decision path taken by an input as an ordered list of indices
  # of the neurons that were evaluated.
  # See: http://scikit-learn.org/stable/auto_examples/tree/plot_unveil_tree_structure.html
  n_nodes = estimator.tree_.node_count
  feature = estimator.tree_.feature

  # First let's retrieve the decision path of each sample. The decision_path
  # method allows to retrieve the node indicator functions. A non zero element of
  # indicator matrix at the position (i, j) indicates that the sample i goes
  # through the node j.
  X_test = [inp]
  node_indicator = estimator.decision_path(X_test)
  # Similarly, we can also have the leaves ids reached by each sample.
  leaf_id = estimator.apply(X_test)
  # Now, it's possible to get the tests that were used to predict a sample or
  # a group of samples. First, let's make it for the sample.
  node_index = node_indicator.indices[node_indicator.indptr[0]:
                                      node_indicator.indptr[1]]
  neuron_ids = []
  for node_id in node_index:
    if leaf_id[0] == node_id:
        continue
    neuron_ids.append(feature[node_id])
  return neuron_ids

def get_suffix_cluster(neuron_ids, neuron_sig,suffixes=train_suffixes):
  # Get the cluster of inputs that such that all inputs in the cluster
  # have provided on/off signature for the provided neurons.
  #
  # The returned cluster is an array of indices (into mnist.train.images).
  return np.where((suffixes[:, neuron_ids] == neuron_sig).all(axis=1))[0]

def is_consistent_cluster(cluster, predictions):
  # Check if all inputs within the cluster have the same prediction.
  # 'cluster' is an array of input ids.
  pred = predictions[cluster[0]]
  for i in cluster:
    if predictions[i] != pred:
      return False
  return True



def get_invariant_inp(estimator, ref_id, suffixes):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - inp: reference input, shape <784,>
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist_inp_images[ref_id]
  ref_suffix = suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig,suffixes)
  imgs = []
  cnt = 0
  for indx1 in range(0,len(cluster)):
    img = mnist.train.images(cluster[indx1])
    fnd = 1
    for i in range(0,len(img)):
      if (ref_img[i] != img[i]):
        fnd = 0
        break
    if (fnd == 1):
        ref_id = cnt
    cnt = cnt + 1
    imgs.append(img)

  imgs_suffixes = fingerprint_signature(imgs,t_fc2)
  ref_suffix = imgs_suffixes[ref_id]
  print('PREFIX',ref_suffix)
  neuron_ids = get_decision_path(estimator, ref_suffix)
  print('NEURON IDS',neuron_ids)
  neuron_sig = ref_suffix[neuron_ids]
  print('NEURON SIGNATURE',neuron_sig)
  cluster = get_suffix_cluster(neuron_ids, neuron_sig,imgs_suffixes)

  return cluster, neuron_ids, neuron_sig

def get_invariant(estimator, ref_id):
  # Returns an invariant found w.r.t. the provided reference input
  # Args
  #  - ref_id: Index (into mnist.train.images) of the reference input
  # Returns:
  #  - cluster: Indices of training inputs that satisfy the invariant
  #  - neuron_id: A list of neurons such that all inputs that agree with
  #    the reference input on the on/off status of these neurons have the
  #    same prediction as the reference input.
  ref_img = mnist.train.images[ref_id]
  ref_suffix = train_suffixes[ref_id]
  neuron_ids = get_decision_path(estimator, ref_suffix)
  neuron_sig = ref_suffix[neuron_ids]
  cluster = get_suffix_cluster(neuron_ids, neuron_sig)
  return cluster, neuron_ids, neuron_sig


def get_all_invariants(estimator):
  # Returns a dictionary mapping each decision tree prediction class
  # to a list of invariants. Each invariant is specified as a triple:
  # - neuron ids
  # - neuron signature (for the neuron ids)
  # - number of training samples that hit it
  # The neuron ids and neuron signature can be supplied to get_suffix_cluster
  # to obtain the cluster of training instances that hit the invariant.
  def is_leaf(node):
    return estimator.tree_.children_left[node] == estimator.tree_.children_right[node]

  def left_child(node):
    return estimator.tree_.children_left[node]

  def right_child(node):
    return estimator.tree_.children_right[node]

  def get_all_paths_rec(node):
    # Returns a list of triples corresponding to paths
    # in the decision tree. Each triple consists of
    # - neurons encountered along the path
    # - signature along the path
    # - prediction class at the leaf
    # - number of training samples that hit the path
    # The prediction class and number of training samples
    # are set to -1 when the leaf is "impure".
    feature = estimator.tree_.feature
    threshold = estimator.tree_.threshold
    if is_leaf(node):
      values = estimator.tree_.value[node][0]
      if len(np.where(values != 0)[0]) == 1:
        cl = estimator.classes_[np.where(values != 0)[0][0]]
        nsamples = estimator.tree_.n_node_samples[node]
      else:
        # impure node
        cl = -1
        nsamples = -1
      return [[[], [], cl, nsamples]]
    # If it is not a leaf both left and right childs must exist
   # paths = [[[feature[node]] + p[0], [0] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
   # paths += [[[feature[node]] + p[0], [1] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    paths = [[[feature[node]] + p[0],['<='] + [threshold[node]] + p[1], p[2], p[3]] for p in get_all_paths_rec(left_child(node))]
    paths += [[[feature[node]] + p[0],['>'] + [threshold[node]] + p[1], p[2], p[3]] for p in get_all_paths_rec(right_child(node))]
    return paths
  paths =  get_all_paths_rec(0)
  print("Obtained all paths")
  invariants = {}
  for p in tqdm(paths):
    neuron_ids, neuron_sig, cl, nsamples = p
    if cl not in invariants:
      invariants[cl] = []
    # cluster = get_suffix_cluster(neuron_ids, neuron_sig)
    invariants[cl].append([neuron_ids, neuron_sig, nsamples])
  for cl in invariants.keys():
    invariants[cl] = sorted(invariants[cl], key=operator.itemgetter(2), reverse=True)
  return invariants


def describe_cluster(cluster, neuron_ids, show_samples=False):
  neuron_sig = train_suffixes[cluster[0]][neuron_ids]
  print("Num neurons in invariant", len(neuron_ids))
  print("Neuron id and signature")

  for i in range(0,len(neuron_ids)):
    print("id:", neuron_ids[i], "sig:", neuron_sig[i])

  print("Cluster size: ", len(cluster))
  print("Num misclassified", len([i for i in cluster if is_misclassified(i)]))
  if show_samples:
    for i in range(10):
      images = []
      for j in range(10):
        if 10*i + j >= len(cluster):
          break
        images.append(mnist_to_pil_img(mnist.train.images[cluster[10*i+j]]))
      if len(images) > 0:
        show_img(combine(images))

def describe_invariants_all_labels_col(all_invariants,suffixes=train_suffixes,COMMON=False, DEC_PREFX= False):
  invs_sel_neus = []
  invs_sel_sig = []
  Minvs_sel_neus = []
  Minvs_sel_sig = []
  Minvs_sel_supp = []
  invs_sel_supp = []
  invs_sel_cl = []
  Minvs_sel_cl = []

  corr_invs_sel_labs = []
  corr_invs_sel_neus = []
  corr_invs_sel_sig = []
  corr_invs_sel_supp = []

  for cl, invs in all_invariants.items():

    for indx in range (0, len(invs)):
      inv = invs[indx]


     # print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2])

    # MISCLASSIFICATION PATTTERN FOR IDEAL LABELs
      if (cl == 1000):
          Minvs_sel_cl.append(cl)
          Minvs_sel_neus.append(inv[0])
          Minvs_sel_sig.append(inv[1])
          Minvs_sel_supp.append(inv[2])

      # MISCLASSIFICATION PATTTERNs FOR ACTUAL LABELS
      if (cl >= 10):
        invs_sel_neus.append(inv[0])
        invs_sel_sig.append(inv[1])
        invs_sel_supp.append(inv[2])
        invs_sel_cl.append(cl)


      #if ((cl >= 0) ): #and (cl not in corr_invs_sel_labs)):
      if ((cl >= 0) and (cl < 10)):
          corr_invs_sel_labs.append(cl)
          corr_invs_sel_neus.append(inv[0])
          corr_invs_sel_sig.append(inv[1])
          corr_invs_sel_supp.append(inv[2])



  return (Minvs_sel_neus, Minvs_sel_sig, Minvs_sel_supp, Minvs_sel_cl, invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp)


def describe_invariants_all_labels(all_invariants,suffixes=train_suffixes,COMMON=False, DEC_PREFX= False):
  invs_sel_neus = []
  invs_sel_sig = []
  invs_sel_supp = []
  corr_invs_sel_labs = []
  corr_invs_sel_neus = []
  corr_invs_sel_sig = []
  corr_invs_sel_supp = []

 # print("PRINTING PURE RULES WITH SUPPORT MORE THAN 100 FOR EVERY LABEL:");
  supp = 0
  for cl, invs in all_invariants.items():

    for indx in range (0, len(invs)):
      inv = invs[indx]
      if (inv[2] <= 10):
        continue
      #print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2])
      print("Class:", cl, "Support:",inv[2])
      if (cl == -1000):
        if (inv[2] > ((10/100) * FAIL_P)):
          invs_sel_neus.append(inv[0])
          invs_sel_sig.append(inv[1])
          invs_sel_supp.append(inv[2])
          supp = supp + inv[2]

      if ((cl >= 0) and (cl not in corr_invs_sel_labs)):
          corr_invs_sel_labs.append(cl)
          corr_invs_sel_neus.append(inv[0])
          corr_invs_sel_sig.append(inv[1])
          corr_invs_sel_supp.append(inv[2])

      continue

      cls = get_suffix_cluster(inv[0],inv[1],suffixes)

      neurons = inv[0]
      signature = inv[1]

      if (len(cls) <= 10):
         continue

     # fail_cnt = []
     # fail_label = []
     # pass_cnt = []
     # for ind in range(0,len(cls)):
     #   inp_indx = cls[ind]
     #   if (FP[inp_indx] == 0):
     #     fail_cnt.append(inp_indx)
     #     fail_label.append(IDEAL[inp_indx])
     #   else:
     #     pass_cnt.append(inp_indx)

      #fail_perc = (fail_cnt/len(cls))
      #if (inv[2] == 243):
      #print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2],",Pass:",[i for i in pass_cnt],",Mis:",[i for i in fail_cnt] , ",Label:",[i for i in fail_label]);

      print("Class:", cl, ", Rule:(neurons:",inv[0],",signature:",inv[1],"),Support:",inv[2])
      #if (indx > 0):
      #  break
     # for ind in range(0, len(cls)):
     #    print(cls[ind])

     # break
      if (COMMON == True):
          common_nodes(cls,suffixes)

      if (DEC_PREFX == True):
          decision_prefs(cls,suffixes)

  return (invs_sel_neus, invs_sel_sig, invs_sel_supp, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp)


def common_nodes(cls,suffixes):
    cnt = 0
    common = np.zeros(10,dtype=int)
    prev = np.zeros(10,dtype=int)

    for indx in range(0, len(cls)):
        i = cls[indx]
        cnt = cnt + 1
        for j in range(0,len(suffixes[i])):
          if (common[j] == -1):
             continue
          if ((indx != 0) and (suffixes[i][j] != prev[j])):
             common[j] = -1
          else:
             common[j] = suffixes[i][j]
          prev[j] = suffixes[i][j]


    print('COMMON NODES IN CLUSTER for CLASS:',cl,cnt)
    com = []
    for k in range(0,len(common)):
        if (common[k] != -1):
           com.append((k,common[k]))
    print(com)

    return

def decision_prefs(cls,suffixes):
    images = mnist.train.images
    imgsCom = []
    imgs = []
    for indx in range(0, len(cls)):
        print('IMG:')
        print(list(zip(images[cls[indx]])))
        imgs.append(images[cls[indx]])
        imgsCom.append(images[cls[indx]])

    dec_prefixes= fingerprint_signature(imgs,layer)
    prefixes = []
    for indx in range(0,len(dec_prefixes)):
       dec_pref = dec_prefixes[indx]

       match = 0
       for indx1 in range(0, len(prefixes)):
          match = 1
          for i in range(0,len(prefixes[indx1])):
             if (dec_pref[i] != prefixes[indx1][i]):
                match = 0
                break
          if (match == 1):
             break

       if (match == 0):
          prefixes.append(dec_pref)

    print('DECISION PREFIXES IN CLUSTER for CLASS:',cl,cnt)
    for k in range(0,len(prefixes)):
      print(prefixes[k])

    return


In [ ]:
#invariants = get_all_invariants(basic_estimator)
#(Ainvs_sel_neus, Ainvs_sel_sig, Ainvs_sel_supp, Acorr_invs_sel_labs, Acorr_invs_sel_neus, Acorr_invs_sel_sig, Acorr_invs_sel_supp) = describe_invariants_all_labels(invariants)
#print("ACTIVATION PATTERNS:")
#for indx in range(0, len(Ainvs_sel_neus)):
#  print(Ainvs_sel_neus[indx], Ainvs_sel_sig[indx], Ainvs_sel_supp[indx])
#for indx in range(0, len(Acorr_invs_sel_labs)):
#  print(Acorr_invs_sel_labs[indx], Acorr_invs_sel_neus[indx], Acorr_invs_sel_sig[indx], Acorr_invs_sel_supp[indx])


#invariants = get_all_invariants(basic_estimator1)
#(invs_sel_neus, invs_sel_sig, invs_sel_supp, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp) = describe_invariants_all_labels(invariants)

corr_invs_sel_lay1 = []
corr_invs_sel_lay1a = []
corr_invs_sel_lay2 = []
corr_invs_sel_lay2a = []
corr_invs_sel_lay3 = []
corr_invs_sel_lay3a = []

invariants_VAL1 = get_all_invariants(basic_estimator1)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs1, corr_invs_sel_neus1, corr_invs_sel_sig1, corr_invs_sel_supp1) = describe_invariants_all_labels_col(invariants_VAL1)

print("\n CORRECT PATTERNS VAL 1:\n")
for indx in range(0, len(corr_invs_sel_neus1)):
   corr_invs_sel_lay1.append(1)
   print(corr_invs_sel_labs1[indx], corr_invs_sel_neus1[indx], corr_invs_sel_sig1[indx], corr_invs_sel_supp1[indx])

invariants_ACT1 = get_all_invariants(basic_estimator1a)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs1a, corr_invs_sel_neus1a, corr_invs_sel_sig1a, corr_invs_sel_supp1a) = describe_invariants_all_labels_col(invariants_ACT1)

print("\n CORRECT PATTERNS ACT 1:\n")
for indx in range(0, len(corr_invs_sel_neus1a)):
   corr_invs_sel_lay1a.append(1)
   print(corr_invs_sel_labs1a[indx], corr_invs_sel_neus1a[indx], corr_invs_sel_sig1a[indx], corr_invs_sel_supp1a[indx])


invariants_VAL2 = get_all_invariants(basic_estimator2)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs2, corr_invs_sel_neus2, corr_invs_sel_sig2, corr_invs_sel_supp2) = describe_invariants_all_labels_col(invariants_VAL2)

print("\n CORRECT PATTERNS VAL 2:\n")
for indx in range(0, len(corr_invs_sel_neus2)):
   corr_invs_sel_lay2.append(2)
   print(corr_invs_sel_labs2[indx], corr_invs_sel_neus2[indx], corr_invs_sel_sig2[indx], corr_invs_sel_supp2[indx])

invariants_ACT2 = get_all_invariants(basic_estimator2a)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs2a, corr_invs_sel_neus2a, corr_invs_sel_sig2a, corr_invs_sel_supp2a) = describe_invariants_all_labels_col(invariants_ACT2)

print("\n CORRECT PATTERNS ACT 2:\n")
for indx in range(0, len(corr_invs_sel_neus2a)):
   corr_invs_sel_lay2a.append(2)
   print(corr_invs_sel_labs2a[indx], corr_invs_sel_neus2a[indx], corr_invs_sel_sig2a[indx], corr_invs_sel_supp2a[indx])

invariants_VAL3 = get_all_invariants(basic_estimator3)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs3, corr_invs_sel_neus3, corr_invs_sel_sig3, corr_invs_sel_supp3) = describe_invariants_all_labels_col(invariants_VAL3)

print("\n CORRECT PATTERNS VAL 3:\n")
for indx in range(0, len(corr_invs_sel_neus3)):
   corr_invs_sel_lay3.append(3)
   print(corr_invs_sel_labs3[indx], corr_invs_sel_neus3[indx], corr_invs_sel_sig3[indx], corr_invs_sel_supp3[indx])

invariants_ACT3 = get_all_invariants(basic_estimator3a)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, corr_invs_sel_labs3a, corr_invs_sel_neus3a, corr_invs_sel_sig3a, corr_invs_sel_supp3a) = describe_invariants_all_labels_col(invariants_ACT3)

print("\n CORRECT PATTERNS ACT 3:\n")
for indx in range(0, len(corr_invs_sel_neus3a)):
   corr_invs_sel_lay3a.append(3)
   print(corr_invs_sel_labs3a[indx], corr_invs_sel_neus3a[indx], corr_invs_sel_sig3a[indx], corr_invs_sel_supp3a[indx])


Obtained all paths


100%|██████████| 350/350 [00:00<00:00, 710211.13it/s]



 CORRECT PATTERNS VAL 1:

8 [118, 30, 49, 28, 99, 25, 45, 53, 54, 0, 33, 44] ['<=', 6.1527440547943115, '<=', 24.32576274871826, '<=', 14.000247478485107, '<=', 9.47304916381836, '<=', -28.7006893157959, '>', -6.378535985946655, '>', -7.638356447219849, '<=', -17.374003410339355, '<=', 7.717164754867554, '>', -41.77583312988281, '<=', 29.031685829162598, '<=', 12.687620162963867] 263
8 [118, 30, 49, 28, 82, 27, 32, 25, 13] ['<=', 6.1527440547943115, '<=', 24.32576274871826, '<=', 14.000247478485107, '>', 9.47304916381836, '<=', 0.9486368894577026, '<=', 1.863332986831665, '>', -11.323588371276855, '>', -2.756648063659668, '<=', -1.3389196395874023] 51
8 [118, 30, 49, 28, 99, 13, 105, 99, 109, 50, 53, 88, 11, 35] ['<=', 6.1527440547943115, '<=', 24.32576274871826, '<=', 14.000247478485107, '<=', 9.47304916381836, '>', -28.7006893157959, '<=', -4.494465589523315, '<=', 12.000867366790771, '<=', 3.7376177310943604, '<=', -1.9472768306732178, '>', 0.8475848436355591, '<=', -17.06834602355

100%|██████████| 556/556 [00:00<00:00, 733113.18it/s]


 CORRECT PATTERNS ACT 1:

1 [118, 42, 61, 15, 45, 75, 110, 85, 107] ['>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5] 505
1 [118, 14, 22, 54, 13, 6, 24, 110, 86] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5] 32
1 [118, 42, 61, 15, 45, 75, 77] ['>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5] 13
1 [118, 42, 61, 15, 59] ['>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5] 7
1 [118, 14, 22, 54, 13, 26, 34, 68, 62] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5] 6
1 [118, 42, 61, 15, 45, 75, 110, 85, 107, 76] ['>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5] 5
1 [118, 14, 22, 28, 54, 111, 59, 109] ['<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5] 4
1 [118, 14, 22, 54, 13, 26, 96, 23, 110, 28, 33, 58, 86, 49] ['<=', 0.5, '<=', 0.5

...


9 [118, 14, 19, 79, 55, 99, 13, 8, 46, 31, 20, 18] ['<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5] 255
9 [118, 42, 13, 110, 90] ['>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5] 51
9 [118, 14, 19, 79, 110, 23, 5, 109, 20, 0, 48] ['<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5] 43
9 [118, 14, 19, 79, 55, 105, 43, 99, 12, 74] ['<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5] 19
9 [118, 14, 22, 54, 49, 46, 109, 29, 23] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5] 14
9 [118, 14, 22, 54, 13, 26, 96, 23, 110, 37] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5] 13
9 [118, 14, 22, 54, 13, 26, 34, 122, 74, 8, 64, 114] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>',

100%|██████████| 116/116 [00:00<00:00, 161587.27it/s]



 CORRECT PATTERNS VAL 3:

4 [1, 0, 6, 3, 4, 7, 9] ['<=', 10.163049697875977, '<=', 9.073975086212158, '<=', 10.416686534881592, '<=', 12.274341583251953, '>', 11.868844509124756, '<=', 13.65552806854248, '<=', 16.086463928222656] 474
4 [1, 0, 6, 3, 4, 9, 2, 7, 8, 5, 1, 3, 4, 4, 4] ['<=', 10.163049697875977, '<=', 9.073975086212158, '<=', 10.416686534881592, '<=', 12.274341583251953, '<=', 11.868844509124756, '<=', 6.868115663528442, '<=', 8.186288356781006, '<=', 5.939865827560425, '<=', 5.550639390945435, '<=', 6.68888783454895, '<=', 1.6966525614261627, '<=', 6.66979455947876, '>', 1.3131107091903687, '>', 8.883097171783447, '<=', 11.127149105072021] 6
4 [1, 0, 6, 3, 4, 9, 2, 7, 8, 5, 1, 3, 4, 4, 8] ['<=', 10.163049697875977, '<=', 9.073975086212158, '<=', 10.416686534881592, '<=', 12.274341583251953, '<=', 11.868844509124756, '<=', 6.868115663528442, '<=', 8.186288356781006, '<=', 5.939865827560425, '<=', 5.550639390945435, '<=', 6.68888783454895, '<=', 1.6966525614261627, '<=', 6.

100%|██████████| 128/128 [00:00<00:00, 371536.96it/s]


 CORRECT PATTERNS ACT 3:

4 [0, 6, 2, 1, 4, 9, 7, 3, 8] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5] 2
4 [0, 6, 2, 1, 4, 8, 5, 3, 7, 9] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5] 1
4 [0, 6, 2, 1, 4, 9, 7, 8, 3, 5] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
4 [0, 6, 2, 1, 4, 9, 7, 8, 3] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
4 [0, 6, 2, 4, 1, 7, 8, 9] ['<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
4 [0, 6, 2, 4, 1, 7, 8] ['<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
4 [0, 6, 1, 8, 2, 9, 5] ['<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5] 1
9 [0, 6, 2, 1, 4, 8, 5, 3, 7, 9] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 

In [ ]:
#invariants = get_all_invariants(basic_estimator)
#(Ainvs_sel_neus, Ainvs_sel_sig, Ainvs_sel_supp, Acorr_invs_sel_labs, Acorr_invs_sel_neus, Acorr_invs_sel_sig, Acorr_invs_sel_supp) = describe_invariants_all_labels(invariants)
#print("ACTIVATION PATTERNS:")
#for indx in range(0, len(Ainvs_sel_neus)):
#  print(Ainvs_sel_neus[indx], Ainvs_sel_sig[indx], Ainvs_sel_supp[indx])
#for indx in range(0, len(Acorr_invs_sel_labs)):
#  print(Acorr_invs_sel_labs[indx], Acorr_invs_sel_neus[indx], Acorr_invs_sel_sig[indx], Acorr_invs_sel_supp[indx])


#invariants = get_all_invariants(basic_estimator1)
#(invs_sel_neus, invs_sel_sig, invs_sel_supp, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp) = describe_invariants_all_labels(invariants)

Rcorr_invs_sel_lay1 = []
Rcorr_invs_sel_lay1a = []
Rcorr_invs_sel_lay2 = []
Rcorr_invs_sel_lay2a = []
Rcorr_invs_sel_lay3 = []
Rcorr_invs_sel_lay3a = []

Rinvariants_VAL1 = get_all_invariants(Rbasic_estimator1)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, Rcorr_invs_sel_labs1, Rcorr_invs_sel_neus1, Rcorr_invs_sel_sig1, Rcorr_invs_sel_supp1) = describe_invariants_all_labels_col(Rinvariants_VAL1)

print("\n CORRECT PATTERNS VAL 1:\n")
for indx in range(0, len(Rcorr_invs_sel_neus1)):
   Rcorr_invs_sel_lay1.append(1)
   print(Rcorr_invs_sel_labs1[indx], Rcorr_invs_sel_neus1[indx], Rcorr_invs_sel_sig1[indx], Rcorr_invs_sel_supp1[indx])

Rinvariants_ACT1 = get_all_invariants(Rbasic_estimator1a)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, Rcorr_invs_sel_labs1a, Rcorr_invs_sel_neus1a, Rcorr_invs_sel_sig1a, Rcorr_invs_sel_supp1a) = describe_invariants_all_labels_col(Rinvariants_ACT1)

print("\n CORRECT PATTERNS ACT 1:\n")
for indx in range(0, len(Rcorr_invs_sel_neus1a)):
   Rcorr_invs_sel_lay1a.append(1)
   print(Rcorr_invs_sel_labs1a[indx], Rcorr_invs_sel_neus1a[indx], Rcorr_invs_sel_sig1a[indx], Rcorr_invs_sel_supp1a[indx])


Rinvariants_VAL2 = get_all_invariants(Rbasic_estimator2)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, Rcorr_invs_sel_labs2, Rcorr_invs_sel_neus2, Rcorr_invs_sel_sig2, Rcorr_invs_sel_supp2) = describe_invariants_all_labels_col(Rinvariants_VAL2)

print("\n CORRECT PATTERNS VAL 2:\n")
for indx in range(0, len(Rcorr_invs_sel_neus2)):
   Rcorr_invs_sel_lay2.append(2)
   print(Rcorr_invs_sel_labs2[indx], Rcorr_invs_sel_neus2[indx], Rcorr_invs_sel_sig2[indx], Rcorr_invs_sel_supp2[indx])

Rinvariants_ACT2 = get_all_invariants(Rbasic_estimator2a)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, Rcorr_invs_sel_labs2a, Rcorr_invs_sel_neus2a, Rcorr_invs_sel_sig2a, Rcorr_invs_sel_supp2a) = describe_invariants_all_labels_col(Rinvariants_ACT2)

print("\n CORRECT PATTERNS ACT 2:\n")
for indx in range(0, len(Rcorr_invs_sel_neus2a)):
   Rcorr_invs_sel_lay2a.append(2)
   print(Rcorr_invs_sel_labs2a[indx], Rcorr_invs_sel_neus2a[indx], Rcorr_invs_sel_sig2a[indx], Rcorr_invs_sel_supp2a[indx])


Rinvariants_VAL3 = get_all_invariants(Rbasic_estimator3)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, Rcorr_invs_sel_labs3, Rcorr_invs_sel_neus3, Rcorr_invs_sel_sig3, Rcorr_invs_sel_supp3) = describe_invariants_all_labels_col(Rinvariants_VAL3)

print("\n CORRECT PATTERNS VAL 3:\n")
for indx in range(0, len(Rcorr_invs_sel_neus3)):
   Rcorr_invs_sel_lay3.append(3)
   print(Rcorr_invs_sel_labs3[indx], Rcorr_invs_sel_neus3[indx], Rcorr_invs_sel_sig3[indx], Rcorr_invs_sel_supp3[indx])

Rinvariants_ACT3 = get_all_invariants(Rbasic_estimator3a)
(Minvs_sel_neus1, Minvs_sel_sig1, Minvs_sel_sup1, Minvs_sel_cl1,invs_sel_neus, invs_sel_sig, invs_sel_supp, invs_sel_cl, Rcorr_invs_sel_labs3a, Rcorr_invs_sel_neus3a, Rcorr_invs_sel_sig3a, Rcorr_invs_sel_supp3a) = describe_invariants_all_labels_col(Rinvariants_ACT3)

print("\n CORRECT PATTERNS ACT 3:\n")
for indx in range(0, len(Rcorr_invs_sel_neus3a)):
   Rcorr_invs_sel_lay3a.append(3)
   print(Rcorr_invs_sel_labs3a[indx], Rcorr_invs_sel_neus3a[indx], Rcorr_invs_sel_sig3a[indx], Rcorr_invs_sel_supp3a[indx])




Obtained all paths


100%|██████████| 463/463 [00:00<00:00, 772766.71it/s]


 CORRECT PATTERNS VAL 1:

8 [118, 52, 49, 29, 82, 25, 70, 66, 117, 35, 125, 101] ['<=', 3.0871529579162598, '<=', 9.906914234161377, '<=', 10.800707340240479, '<=', 4.310592174530029, '<=', 7.4074931144714355, '>', -3.30419659614563, '<=', -18.583422660827637, '<=', 3.3646847009658813, '>', -11.567915439605713, '<=', 19.921862602233887, '<=', -15.263777256011963, '<=', 10.598014831542969] 343
8 [118, 52, 49, 29, 82, 25, 99, 96, 52, 55, 46, 23, 105, 101, 46, 35, 27, 35] ['<=', 3.0871529579162598, '<=', 9.906914234161377, '<=', 10.800707340240479, '<=', 4.310592174530029, '<=', 7.4074931144714355, '<=', -3.30419659614563, '>', -34.30601692199707, '<=', 14.452005386352539, '<=', -2.838945746421814, '<=', -0.4035930037498474, '<=', 2.4229977130889893, '<=', 20.69230079650879, '<=', 31.981432914733887, '<=', -17.771254539489746, '<=', -14.676931381225586, '<=', 20.17630386352539, '<=', 3.9017538130283356, '>', -30.596739768981934] 58
8 [118, 52, 49, 29, 82, 25, 70, 19, 3, 108, 47, 76] ['<=


9 [118, 52, 49, 29, 13, 55, 99, 124, 6, 96, 18, 56, 51, 93, 125, 55] ['<=', 3.0871529579162598, '<=', 9.906914234161377, '<=', 10.800707340240479, '>', 4.310592174530029, '<=', -5.311701774597168, '>', 2.0427396297454834, '<=', 12.469228267669678, '>', -8.828258514404297, '>', 1.316409558057785, '<=', 16.780364990234375, '<=', 6.4265183210372925, '>', -13.894166946411133, '>', -33.18592071533203, '<=', -13.112791538238525, '>', -14.206780433654785, '>', 7.029850721359253] 1
9 [118, 52, 49, 29, 13, 6, 57, 14, 94, 99, 10, 114, 44, 70, 10, 117, 17] ['<=', 3.0871529579162598, '<=', 9.906914234161377, '<=', 10.800707340240479, '>', 4.310592174530029, '>', -5.311701774597168, '>', 10.512630462646484, '<=', 6.400945663452148, '<=', 19.027442932128906, '<=', -28.923681259155273, '<=', 23.51803970336914, '>', -51.55147743225098, '<=', 22.38546657562256, '<=', 7.0204572677612305, '<=', -9.042710781097412, '<=', 5.032315731048584, '>', 25.178672790527344, '>', 12.891462326049805] 1
9 [118, 52, 4

100%|██████████| 723/723 [00:00<00:00, 127834.15it/s]

...

100%|██████████| 216/216 [00:00<00:00, 354171.10it/s]


 CORRECT PATTERNS VAL 3:

6 [1, 2, 4, 3, 9, 6, 0, 5, 8, 1, 3] ['<=', 10.440127849578857, '<=', 10.167909145355225, '<=', 8.207056045532227, '<=', 9.173921585083008, '<=', 6.868115663528442, '>', 7.394034147262573, '<=', 10.312762260437012, '<=', 11.753541469573975, '<=', 9.07082200050354, '<=', 3.630765199661255, '<=', -1.2081783264875412] 518
6 [1, 2, 4, 3, 9, 6, 0, 5, 8, 7, 7, 6, 0, 8] ['<=', 10.440127849578857, '<=', 10.167909145355225, '<=', 8.207056045532227, '<=', 9.173921585083008, '<=', 6.868115663528442, '<=', 7.394034147262573, '<=', -1.067255675792694, '<=', 4.826073884963989, '<=', -2.425419569015503, '>', 39.38429832458496, '<=', 66.91107177734375, '>', -18.366581916809082, '<=', -10.107964992523193, '<=', -7.635807991027832] 81
6 [1, 2, 4, 9, 6] ['<=', 10.440127849578857, '<=', 10.167909145355225, '>', 8.207056045532227, '<=', 12.109760284423828, '>', 14.967638492584229] 8
6 [1, 2, 4, 3, 9, 6, 0, 5, 8, 7, 7, 1, 2, 3, 9] ['<=', 10.440127849578857, '<=', 10.167909145355225

1
7 [1, 2, 4, 3, 9, 5, 0, 1, 9, 9] ['<=', 10.440127849578857, '<=', 10.167909145355225, '<=', 8.207056045532227, '<=', 9.173921585083008, '>', 6.868115663528442, '<=', 7.160873651504517, '<=', 9.904133558273315, '<=', -3.6505740880966187, '<=', 7.168414354324341, '>', 7.1114277839660645] 1
7 [1, 2, 4, 3, 5, 8, 5] ['<=', 10.440127849578857, '<=', 10.167909145355225, '<=', 8.207056045532227, '>', 9.173921585083008, '<=', 10.01953125, '<=', 10.87579870223999, '<=', -38.50971603393555] 1
7 [1, 2, 4, 3, 5, 8, 5, 4, 1, 1] ['<=', 10.440127849578857, '<=', 10.167909145355225, '<=', 8.207056045532227, '>', 9.173921585083008, '<=', 10.01953125, '<=', 10.87579870223999, '>', -38.50971603393555, '<=', -6.1693878173828125, '>', 3.6181310415267944, '<=', 4.034226059913635] 1
7 [1, 2, 4, 3, 5, 8, 5, 4] ['<=', 10.440127849578857, '<=', 10.167909145355225, '<=', 8.207056045532227, '>', 9.173921585083008, '<=', 10.01953125, '<=', 10.87579870223999, '>', -38.50971603393555, '>', -6.1693878173828125] 1
7 

100%|██████████| 138/138 [00:00<00:00, 526194.50it/s]


 CORRECT PATTERNS ACT 3:

4 [0, 6, 1, 4, 9, 8, 7] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5] 2
4 [0, 6, 1, 4, 5, 9, 8, 3, 7, 2] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5] 1
4 [0, 6, 1, 4, 9, 8, 7, 2, 5, 3] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
4 [0, 6, 1, 4, 9, 8, 7, 2, 5] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
4 [0, 6, 1, 4, 9, 8, 7, 2] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5] 1
4 [0, 6, 1, 4, 9, 7, 5, 8, 3, 2] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
4 [0, 6, 1, 2, 7, 3, 8, 4] ['<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
4 [0, 6, 8, 1, 2, 9, 4] ['<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '

In [ ]:
#invariants = get_all_invariants(basic_estimator)
#(Ainvs_sel_neus, Ainvs_sel_sig, Ainvs_sel_supp, Acorr_invs_sel_labs, Acorr_invs_sel_neus, Acorr_invs_sel_sig, Acorr_invs_sel_supp) = describe_invariants_all_labels(invariants)
#print("ACTIVATION PATTERNS:")
#for indx in range(0, len(Ainvs_sel_neus)):
#  print(Ainvs_sel_neus[indx], Ainvs_sel_sig[indx], Ainvs_sel_supp[indx])
#for indx in range(0, len(Acorr_invs_sel_labs)):
#  print(Acorr_invs_sel_labs[indx], Acorr_invs_sel_neus[indx], Acorr_invs_sel_sig[indx], Acorr_invs_sel_supp[indx])


#invariants = get_all_invariants(basic_estimator1)
#(invs_sel_neus, invs_sel_sig, invs_sel_supp, corr_invs_sel_labs, corr_invs_sel_neus, corr_invs_sel_sig, corr_invs_sel_supp) = describe_invariants_all_labels(invariants)

Pcorr_invs_sel_lay1 = []
Pcorr_invs_sel_lay1a = []
Pcorr_invs_sel_lay2 = []
Pcorr_invs_sel_lay2a = []
Pcorr_invs_sel_lay3 = []
Pcorr_invs_sel_lay3a = []

PMinvs_sel_lay1 = []
PMinvs_sel_lay1a = []
PMinvs_sel_lay2 = []
PMinvs_sel_lay2a = []
PMinvs_sel_lay3 = []
PMinvs_sel_lay3a = []

Pinvariants_VAL1 = get_all_invariants(Pbasic_estimator1)
(PMinvs_sel_neus1, PMinvs_sel_sig1, PMinvs_sel_sup1, PMinvs_sel_cl1,Pinvs_sel_neus, Pinvs_sel_sig, Pinvs_sel_supp, Pinvs_sel_cl, Pcorr_invs_sel_labs1, Pcorr_invs_sel_neus1, Pcorr_invs_sel_sig1, Pcorr_invs_sel_supp1) = describe_invariants_all_labels_col(Pinvariants_VAL1)

print("\n CORRECT PATTERNS VAL 1:\n")
for indx in range(0, len(Pcorr_invs_sel_neus1)):
   Pcorr_invs_sel_lay1.append(1)
   print(Pcorr_invs_sel_labs1[indx], Pcorr_invs_sel_neus1[indx], Pcorr_invs_sel_sig1[indx], Pcorr_invs_sel_supp1[indx])

print("\n MISCLASS PATTERNS VAL 1:\n")
for indx in range(0, len(PMinvs_sel_neus1)):
   PMinvs_sel_lay1.append(1)
   print(PMinvs_sel_cl1[indx], PMinvs_sel_neus1[indx], PMinvs_sel_sig1[indx], PMinvs_sel_sup1[indx])

Pinvariants_ACT1 = get_all_invariants(Pbasic_estimator1a)
(PMinvs_sel_neus1a, PMinvs_sel_sig1a, PMinvs_sel_sup1a, PMinvs_sel_cl1a,Pinvs_sel_neus1a, Pinvs_sel_sig1a, Pinvs_sel_supp1a, Pinvs_sel_cl1a, Pcorr_invs_sel_labs1a, Pcorr_invs_sel_neus1a, Pcorr_invs_sel_sig1a, Pcorr_invs_sel_supp1a) = describe_invariants_all_labels_col(Pinvariants_ACT1)

print("\n CORRECT PATTERNS ACT 1:\n")
for indx in range(0, len(Pcorr_invs_sel_neus1a)):
   Pcorr_invs_sel_lay1a.append(1)
   print(Pcorr_invs_sel_labs1a[indx], Pcorr_invs_sel_neus1a[indx], Pcorr_invs_sel_sig1a[indx], Pcorr_invs_sel_supp1a[indx])

print("\n MISCLASS PATTERNS ACT 1:\n")
for indx in range(0, len(PMinvs_sel_neus1a)):
   PMinvs_sel_lay1a.append(1)
   print(PMinvs_sel_cl1a[indx], PMinvs_sel_neus1a[indx], PMinvs_sel_sig1a[indx], PMinvs_sel_sup1a[indx])


Pinvariants_VAL2 = get_all_invariants(Pbasic_estimator2)
(PMinvs_sel_neus2, PMinvs_sel_sig2, PMinvs_sel_sup2, PMinvs_sel_cl2,invs_sel_neus, Pinvs_sel_sig, Pinvs_sel_supp,Pinvs_sel_cl, Pcorr_invs_sel_labs2, Pcorr_invs_sel_neus2, Pcorr_invs_sel_sig2, Pcorr_invs_sel_supp2) = describe_invariants_all_labels_col(Pinvariants_VAL2)

print("\n CORRECT PATTERNS VAL 2:\n")
for indx in range(0, len(Pcorr_invs_sel_neus2)):
   Pcorr_invs_sel_lay2.append(2)
   print(Pcorr_invs_sel_labs2[indx], Pcorr_invs_sel_neus2[indx], Pcorr_invs_sel_sig2[indx], Pcorr_invs_sel_supp2[indx])

print("\n MISCLASS PATTERNS VAL 2:\n")
for indx in range(0, len(PMinvs_sel_neus2)):
   PMinvs_sel_lay2.append(2)
   print(PMinvs_sel_cl2[indx], PMinvs_sel_neus2[indx], PMinvs_sel_sig2[indx], PMinvs_sel_sup2[indx])

Pinvariants_ACT2 = get_all_invariants(Pbasic_estimator2a)
(PMinvs_sel_neus2a, PMinvs_sel_sig2a, PMinvs_sel_sup2a, PMinvs_sel_cl2a,invs_sel_neus, Pinvs_sel_sig, Pinvs_sel_supp, Pinvs_sel_cl, Pcorr_invs_sel_labs2a, Pcorr_invs_sel_neus2a, Pcorr_invs_sel_sig2a, Pcorr_invs_sel_supp2a) = describe_invariants_all_labels_col(Pinvariants_ACT2)

print("\n CORRECT PATTERNS ACT 2:\n")
for indx in range(0, len(Pcorr_invs_sel_neus2a)):
   Pcorr_invs_sel_lay2a.append(2)
   print(Pcorr_invs_sel_labs2a[indx], Pcorr_invs_sel_neus2a[indx], Pcorr_invs_sel_sig2a[indx], Pcorr_invs_sel_supp2a[indx])

print("\n MISCLASS PATTERNS ACT 2:\n")
for indx in range(0, len(PMinvs_sel_neus2a)):
   PMinvs_sel_lay2a.append(2)
   print(PMinvs_sel_cl2a[indx], PMinvs_sel_neus2a[indx], PMinvs_sel_sig2a[indx], PMinvs_sel_sup2a[indx])


Pinvariants_VAL3 = get_all_invariants(Pbasic_estimator3)
(PMinvs_sel_neus3, PMinvs_sel_sig3, PMinvs_sel_sup3, PMinvs_sel_cl3,invs_sel_neus, Pinvs_sel_sig, Pinvs_sel_supp,Pinvs_sel_cl, Pcorr_invs_sel_labs3, Pcorr_invs_sel_neus3, Pcorr_invs_sel_sig3, Pcorr_invs_sel_supp3) = describe_invariants_all_labels_col(Pinvariants_VAL3)

print("\n CORRECT PATTERNS VAL 3:\n")
for indx in range(0, len(Pcorr_invs_sel_neus3)):
   Pcorr_invs_sel_lay3.append(3)
   print(Pcorr_invs_sel_labs3[indx], Pcorr_invs_sel_neus3[indx], Pcorr_invs_sel_sig3[indx], Pcorr_invs_sel_supp3[indx])

print("\n MISCLASS PATTERNS VAL 3:\n")
for indx in range(0, len(PMinvs_sel_neus3)):
   PMinvs_sel_lay3.append(3)
   print(PMinvs_sel_cl3[indx], PMinvs_sel_neus3[indx], PMinvs_sel_sig3[indx], PMinvs_sel_sup3[indx])

Pinvariants_ACT3 = get_all_invariants(Pbasic_estimator3a)
(PMinvs_sel_neus3a, PMinvs_sel_sig3a, PMinvs_sel_sup3a, PMinvs_sel_cl3a,invs_sel_neus, Pinvs_sel_sig, Pinvs_sel_supp, Pinvs_sel_cl, Pcorr_invs_sel_labs3a, Pcorr_invs_sel_neus3a, Pcorr_invs_sel_sig3a, Pcorr_invs_sel_supp3a) = describe_invariants_all_labels_col(Pinvariants_ACT3)

print("\n CORRECT PATTERNS ACT 3:\n")
for indx in range(0, len(Pcorr_invs_sel_neus3a)):
   Pcorr_invs_sel_lay3a.append(3)
   print(Pcorr_invs_sel_labs3a[indx], Pcorr_invs_sel_neus3a[indx], Pcorr_invs_sel_sig3a[indx], Pcorr_invs_sel_supp3a[indx])

print("\n MISCLASS PATTERNS ACT 3:\n")
for indx in range(0, len(PMinvs_sel_neus3a)):
   PMinvs_sel_lay3a.append(3)
   print(PMinvs_sel_cl3a[indx], PMinvs_sel_neus3a[indx], PMinvs_sel_sig3a[indx], PMinvs_sel_sup3a[indx])


Obtained all paths


100%|██████████| 405/405 [00:00<00:00, 422245.37it/s]


 CORRECT PATTERNS VAL 1:

8 [71, 118, 30, 49, 99, 25, 104, 54, 83, 35, 4, 98, 33, 80] ['<=', 1.5181148052215576, '<=', 6.1527440547943115, '<=', 24.35554599761963, '<=', 14.051221370697021, '<=', -28.7006893157959, '>', -6.385160446166992, '<=', -8.616278171539307, '<=', 1.9781556725502014, '<=', 2.5298640727996826, '<=', 12.995212078094482, '<=', 14.060327053070068, '<=', 10.348367691040039, '<=', 29.031685829162598, '>', -38.13593292236328] 319
8 [71, 118, 30, 49, 99, 13, 105, 90, 27, 109, 52, 110, 26, 68, 35] ['<=', 1.5181148052215576, '<=', 6.1527440547943115, '<=', 24.35554599761963, '<=', 14.051221370697021, '>', -28.7006893157959, '<=', -4.4803407192230225, '<=', 12.000867366790771, '<=', -3.7143492698669434, '<=', 17.820690155029297, '<=', 2.4781183004379272, '<=', -7.787455320358276, '<=', 19.771410942077637, '>', 1.484812930226326, '<=', 22.859585762023926, '<=', 17.817041397094727] 53
8 [71, 118, 30, 49, 99, 25, 104, 84, 50, 62, 24] ['<=', 1.5181148052215576, '<=', 6.152744

 ['>', 1.5181148052215576, '<=', -4.121811747550964, '>', -0.15975597314536572, '>', 11.442275047302246, '>', -45.37623977661133, '<=', 19.781274795532227, '<=', 24.8163423538208, '<=', 24.806307792663574, '>', 14.053960800170898] 1
5 [71, 105, 1, 20, 102, 77, 90, 118, 32, 106, 79] ['>', 1.5181148052215576, '>', -4.121811747550964, '<=', 41.43275833129883, '<=', -3.769514799118042, '>', -19.402934074401855, '<=', -1.0230746865272522, '<=', 4.8670045137405396, '<=', -30.311899185180664, '<=', -22.378772258758545, '>', -3.240419924259186, '<=', -33.02551984786987] 1
9 [71, 118, 30, 49, 99, 13, 105, 90, 99, 86, 55, 61, 96, 18, 59, 124] ['<=', 1.5181148052215576, '<=', 6.1527440547943115, '<=', 24.35554599761963, '<=', 14.051221370697021, '>', -28.7006893157959, '<=', -4.4803407192230225, '<=', 12.000867366790771, '>', -3.7143492698669434, '<=', 2.8585214614868164, '>', -6.924541234970093, '>', -15.323460102081299, '<=', 26.85256576538086, '<=', 16.853687286376953, '<=', 6.4265183210372925

100%|██████████| 670/670 [00:00<00:00, 521079.86it/s]

...

2 [71, 118, 19, 79, 25, 54, 26, 33, 122, 4, 21, 66, 52, 64, 8] ['<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5] 1
2 [71, 118, 19, 79, 25, 54, 26, 13, 5, 83, 10, 99, 97, 46, 117, 119, 123] ['<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
2 [71, 118, 19, 79, 55, 105, 37, 45, 84, 50, 123, 110] ['<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5] 1
2 [71, 118, 19, 79, 55, 105, 37, 6, 106, 65, 0] ['<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '>', 0.5] 1
2 [71, 118, 19, 79, 55, 13, 99, 86, 8, 114] ['<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '>', 0.5] 1
2 [71, 105, 28

100%|██████████| 157/157 [00:00<00:00, 327126.54it/s]



 CORRECT PATTERNS VAL 3:

0 [7, 1, 0, 6, 8, 0] ['<=', 32.47653007507324, '<=', 10.163049697875977, '>', 9.073975086212158, '<=', 19.758502960205078, '<=', 7.7694549560546875, '>', 12.555549144744873] 473
0 [7, 1, 0, 6, 8, 0, 6, 2] ['<=', 32.47653007507324, '<=', 10.163049697875977, '>', 9.073975086212158, '<=', 19.758502960205078, '<=', 7.7694549560546875, '<=', 12.555549144744873, '<=', 5.829584836959839, '<=', 3.0108062028884888] 12
0 [7, 1, 0, 6, 3, 4, 9, 2, 8, 5, 7, 1, 3, 6, 8, 4, 9, 5, 1] ['<=', 32.47653007507324, '<=', 10.163049697875977, '<=', 9.073975086212158, '<=', 10.416686534881592, '<=', 12.274341583251953, '<=', 11.885562419891357, '<=', 6.868115663528442, '<=', 8.186288356781006, '<=', 7.0417397022247314, '<=', 6.298015594482422, '<=', 4.0597593784332275, '<=', 1.6966525614261627, '<=', 4.380017280578613, '<=', 5.513231039047241, '<=', 3.0447089672088623, '<=', 1.3131107091903687, '<=', 1.8701781630516052, '<=', 0.5168128311634064, '<=', -13.873618602752686] 2
0 [7, 1, 

100%|██████████| 138/138 [00:00<00:00, 356632.13it/s]


 CORRECT PATTERNS ACT 3:

4 [7, 3, 9, 4, 8, 1] ['>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5] 2
4 [7, 1, 0, 6, 4, 8, 5, 9, 2, 3] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5] 1
4 [7, 1, 0, 6, 4, 9, 8, 2, 3, 5] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
4 [7, 1, 0, 6, 4, 9, 8, 2, 3] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
4 [7, 1, 0, 6, 4, 9, 8, 2] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5] 1
4 [7, 1, 0, 6, 4, 9, 5, 8, 3, 2] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5] 1
4 [7, 1, 0, 6, 8, 2, 9, 4] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '<=', 0.5, '<=', 0.5, '>', 0.5, '>', 0.5] 1
3 [7, 1, 0, 6, 4, 8, 5, 9, 2, 3] ['<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<=', 0.5, '<='

In [ ]:
# INVOKE FOR EVERY CORRECT LABEL
def corr_match_pattern(layer_vals,suff,neuron_ids,neuron_sig):
   max_match_cnt = -1
   for ix in range(0,len(neuron_ids)):
     match_cnt = 0
     oper = -1
     for ind in range(0,len(neuron_ids[ix])):
       if (ind % 2 == 0):
         op = neuron_sig[ix][ind]
         if (op == '<='):
           oper = 0
         else:
           oper = 1
       else:
         v = neuron_ids[ix][ind]
         vsig = neuron_sig[ix][ind]
         val = layer_vals[v]
         # print(v,vsig,val,oper)
         if (oper == 0):
          if (val <= vsig):
            match_cnt = match_cnt + 1
         else:
          if (val > vsig):
            match_cnt = match_cnt + 1
         oper = -1

     if (match_cnt > max_match_cnt):
       max_match_cnt = match_cnt

   return max_match_cnt

def check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3, neuron_lay,neuron_ids,neuron_sig,support=[],supp_thres=-1):

   labs = []
   found = True;
   oper = -1
   layer_vals1 = (layer_vals1).flatten()
   layer_vals2 = (layer_vals2).flatten()
   layer_vals3 = (layer_vals3).flatten()


   for ix in range(0,len(neuron_ids)):
     if (len(support) > 0):
         if (support[ix] <= supp_thres):
           continue
     found = True
     for ind in range(0,len(neuron_ids[ix])):
       if (ind % 2 == 0):
         op = neuron_sig[ix][ind]
         if (op == '<='):
           oper = 0
         else:
           oper = 1
       else:
         v = neuron_ids[ix][ind]
         vsig = neuron_sig[ix][ind]
         val = 0
         if (neuron_lay[ix] == 1):
           val = layer_vals1[v]
         if (neuron_lay[ix] == 2):
           val = layer_vals2[v]
         if (neuron_lay[ix] == 3):
           val = layer_vals3[v]

         #print("oper:",oper,"layer:",neuron_lay[ix],"pattern value:", v, "pattern sig:",vsig, "inp value:",val)

         if (oper == 0):
          if (val > vsig):
            found = False
          #  print('False')
            break
         else:
          if (val <= vsig):
            found = False
           # print('False')
            break
         oper = -1
     if (found == True):
       #break
       labs.append(ix)

   return labs
   #if (found == False):
   #   return -1
   #else:
   #  return ix

def check_pattern_inter(layer_vals,suff,neuron_ids,neuron_sig,VAL = True,ALL=False):

   if (VAL == False):
      if ((suff[:,neuron_ids][0] == neuron_sig).all(axis=0)):
        return True
      else:
        return False


   found = True;
   oper = -1
   layer_vals = (layer_vals).flatten()

   for ix in range(0,len(neuron_ids)):
     found = True
     for ind in range(0,len(neuron_ids[ix])):
       if (ind % 2 == 0):
         op = neuron_sig[ix][ind]
         if (op == '<='):
           oper = 0
         else:
           oper = 1
       else:
         v = neuron_ids[ix][ind]
         vsig = neuron_sig[ix][ind]
         val = layer_vals[v]
         #print(oper, v, vsig, val)
         if (oper == 0):
          if (val > vsig):
            found = False
          #  print('False')
            break
         else:
          if (val <= vsig):
            found = False
           # print('False')
            break
         oper = -1
     if (found == True):
       break

   if (found == False):
      return -1
   else:
     return ix

In [ ]:
def check_pattern(layer_vals,suff,neuron_ids,neuron_sig,VAL = True,ALL=False):
   #layer = 'activation_3'  #layer name, get it from the model summary
   #neuron_ids =  [-1,71,-1,20,-1,1,-1,28,-1,79,-1,29,-1,39,-1,49,-1,42,-1,33,-1,114,-1,117,-1,88,-1,31,-1,36,-1,33,-1,17,-1,117,-1,1,-1,28,-1,97] # neuron ids in the pattern
   #neuron_sig =  ['>',0.435630053,'>',4.067234278,'<=',42.78802872,'<=',20.19964218,'<=',41.11421585,'<=',23.87646389,'<=',15.27161694,'<=',36.27468491,'<=',30.84732342,'<=',40.63782883,'<=',25.80421829,'<=',43.16035843,'<=',38.44324303,'<=',66.91875458,'<=',26.18983078,'<=',16.70914364,'<=',41.36343193,'<=',35.72474098,'<=',35.5690403,'<=',14.66370487,'<=',40.59473419189453]

   if (VAL == False):
      if ((suff[:,neuron_ids][0] == neuron_sig).all(axis=0)):
        return True
      else:
        return False

   found = True;
   #Integer v = null;
	#			Double vsig = null;
	#			String op = null;
	#			int oper = -1;
   oper = -1
   for ind in range(0,len(neuron_ids)):
     if (ind % 2 == 0):
       op = neuron_sig[ind]
       if (op == '<='):
         oper = 0
       else:
         oper = 1
     else:
       v = neuron_ids[ind]
       vsig = neuron_sig[ind]
       val = layer_vals[v]
      # print(v,vsig,val,oper)
       if (oper == 0):
         if (val > vsig):
           found = False
           break
       else:
         if (val <= vsig):
            found = False
            break
       oper = -1

   return found

In [ ]:
#### STRATEGY 1A, CLEAN DATA
strat1a_start_time = time.time()

neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

Cneuron_ids =[]
C_neuron_ids=[]
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []

corr_invs_sel_supp = []
corr_invs_sel_neus = []
corr_invs_sel_sig = []
corr_invs_sel_labs = []
corr_invs_sel_lay = []


for ix in range (0, len(corr_invs_sel_supp1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
  corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
  corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
  corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
  corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])



for ix in range (0, len(corr_invs_sel_supp2)):
  corr_invs_sel_supp.append(corr_invs_sel_supp2[ix])
  corr_invs_sel_neus.append(corr_invs_sel_neus2[ix])
  corr_invs_sel_sig.append(corr_invs_sel_sig2[ix])
  corr_invs_sel_labs.append(corr_invs_sel_labs2[ix])
  corr_invs_sel_lay.append(corr_invs_sel_lay2[ix])


for ix in range (0, len(corr_invs_sel_supp)-1):
    for iy in range(ix +1, len(corr_invs_sel_supp)):
        if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
          temp = corr_invs_sel_supp[iy]
          corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
          corr_invs_sel_supp[ix] = temp
          temp = corr_invs_sel_neus[iy]
          corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
          corr_invs_sel_neus[ix] = temp
          temp = corr_invs_sel_sig[iy]
          corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
          corr_invs_sel_sig[ix] = temp
          temp = corr_invs_sel_labs[iy]
          corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
          corr_invs_sel_labs[ix] = temp
          temp = corr_invs_sel_lay[iy]
          corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
          corr_invs_sel_lay[ix] = temp


for indx in range (0, len(corr_invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(corr_invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(corr_invs_sel_neus[indx])


for indx in range (0, len(corr_invs_sel_sig)):
      Cneuron_sig.append(corr_invs_sel_sig[indx])
      Cneuron_cl.append(corr_invs_sel_labs[indx])
      Cneuron_lay.append(corr_invs_sel_lay[indx])

print("\n CORRECT PATTERNS\n")
for indx in range (0, len(Cneuron_cl)):
  print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


x = copy.deepcopy(val_X)
cnt = 0
correct = 0

labels = (model.predict(x)).argmax(axis=1)
ideals = val_Y




true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
cnt_not_fnd = 0
broke = 0
pois_pat = 0


PASS = 0
PASS_bef = 0
FAIL = 0
FAIL_bef = 0

clean_pass = 0
clean_fail = 0
pois_pass = 0
pois_fail = 0


for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    cnt = cnt + 1
    image = x[inp_cnt]
    label = labels[inp_cnt]

   # inp = image
   # inc_layer = 'dense'  #layer name, get it from the model summary
   # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

    #layer_vals = inc_suff[0]
    #suff = []

    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy][0]))

    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=img
    img_mod = np.expand_dims(img,axis=0)





    inp = image
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []

    inp = image
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]

    inp = image
    inc_layer = 'dense_2'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals3 = inc_suff[0]
    suff = []





    max_lab7 = False
    match = -1
    match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    print("MATCH:", match)
    Same = True
    if (len(match) == 0):
      cnt_not_fnd = cnt_not_fnd + 1
      new_lbl = labels[inp_cnt]
    else:
      if (Cneuron_lay[match[0]] == 3):
        max_lab7 = True
        if (len(match) > 1):
          new_lbl = Cneuron_cl[match[1]]
        else:
          new_lbl = labels[inp_cnt]
      else:
        new_lbl = Cneuron_cl[match[0]]

      Same = True
      prev_lab = -1
      for indx in range(0,len(match)):
         print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
         if (Cneuron_lay[match[indx]] ==3):
           continue
         if (prev_lab != -1 and Cneuron_cl[match[indx]] != prev_lab):
             Same=False
         prev_lab = Cneuron_cl[match[indx]]




    if (new_lbl == ideals[inp_cnt]):
      PASS = PASS + 1
      if (labels[inp_cnt] != ideals[inp_cnt]):
        correct = correct + 1
        if (max_lab7 == True):
          pois_pat = pois_pat + 1
    else:
      # if (Same == False):
      #   print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
      FAIL = FAIL + 1
      if (labels[inp_cnt] == ideals[inp_cnt]):
        broke = broke + 1

print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
print('CORRECTED:', float(correct/cnt)*100)
print('BROKEN:', float(broke/cnt)*100)
print('POIS PAT',pois_pat)
print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

strat1a_time = (time.time() - strat1a_start_time)/ len(x)


Streaming output truncated to the last 5000 lines.
MATCH: [4, 12]
LAYER,LABEL,SUPP 1 4 401
LAYER,LABEL,SUPP 2 4 341
MATCH: []
MATCH: [46, 133]
LAYER,LABEL,SUPP 2 2 20
LAYER,LABEL,SUPP 1 7 5
MATCH: [13, 15]
LAYER,LABEL,SUPP 2 7 319
LAYER,LABEL,SUPP 1 7 297
MATCH: [110, 266]
LAYER,LABEL,SUPP 1 9 6
LAYER,LABEL,SUPP 2 3 2
MATCH: [66, 80]
LAYER,LABEL,SUPP 1 3 12
LAYER,LABEL,SUPP 2 3 9
MATCH: [13, 15]
LAYER,LABEL,SUPP 2 7 319
LAYER,LABEL,SUPP 1 7 297
MATCH: [10, 11]
LAYER,LABEL,SUPP 1 6 375
LAYER,LABEL,SUPP 2 6 375
MATCH: [10, 11]
LAYER,LABEL,SUPP 1 6 375
LAYER,LABEL,SUPP 2 6 375
MATCH: [0, 1]
LAYER,LABEL,SUPP 1 1 506
LAYER,LABEL,SUPP 2 1 504
MATCH: [2, 3]
LAYER,LABEL,SUPP 2 0 436
LAYER,LABEL,SUPP 1 0 436
MATCH: [13, 15]
LAYER,LABEL,SUPP 2 7 319
LAYER,LABEL,SUPP 1 7 297
MATCH: [4, 44]
LAYER,LABEL,SUPP 1 4 401
LAYER,LABEL,SUPP 2 6 20
MATCH: [14, 41]
LAYER,LABEL,SUPP 2 2 319
LAYER,LABEL,SUPP 1 2 23
MATCH: [0, 1]
LAYER,LABEL,SUPP 1 1 506
LAYER,LABEL,SUPP 2 1 504
MATCH: [0, 1]
LAYER,LABEL,SUPP 1

In [ ]:
#### STRATEGY 1b, CLEAN DATA
strat1b_start_time = time.time()

neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

Cneuron_ids =[]
C_neuron_ids=[]
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []

corr_invs_sel_supp = []
corr_invs_sel_neus = []
corr_invs_sel_sig = []
corr_invs_sel_labs = []
corr_invs_sel_lay = []


MCneuron_ids =[]
MC_neuron_ids=[]
MCneuron_sig = []
MCneuron_cl = []
MCneuron_lay = []

Minvs_sel_supp = []
Minvs_sel_neus = []
Minvs_sel_sig = []
Minvs_sel_labs = []
Minvs_sel_lay = []

for ix in range (0, len(corr_invs_sel_supp1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
  corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
  corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
  corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
  corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])



for ix in range (0, len(corr_invs_sel_supp2)):
  corr_invs_sel_supp.append(corr_invs_sel_supp2[ix])
  corr_invs_sel_neus.append(corr_invs_sel_neus2[ix])
  corr_invs_sel_sig.append(corr_invs_sel_sig2[ix])
  corr_invs_sel_labs.append(corr_invs_sel_labs2[ix])
  corr_invs_sel_lay.append(corr_invs_sel_lay2[ix])


for ix in range (0, len(PMinvs_sel_sup1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  Minvs_sel_supp.append(PMinvs_sel_sup1[ix])
  Minvs_sel_neus.append(PMinvs_sel_neus1[ix])
  Minvs_sel_sig.append(PMinvs_sel_sig1[ix])
  Minvs_sel_labs.append(PMinvs_sel_cl1[ix])
  Minvs_sel_lay.append(PMinvs_sel_lay1[ix])



for ix in range (0, len(PMinvs_sel_sup2)):
  Minvs_sel_supp.append(PMinvs_sel_sup2[ix])
  Minvs_sel_neus.append(PMinvs_sel_neus2[ix])
  Minvs_sel_sig.append(PMinvs_sel_sig2[ix])
  Minvs_sel_labs.append(PMinvs_sel_cl2[ix])
  Minvs_sel_lay.append(PMinvs_sel_lay2[ix])




for ix in range (0, len(corr_invs_sel_supp)-1):
    for iy in range(ix +1, len(corr_invs_sel_supp)):
        if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
          temp = corr_invs_sel_supp[iy]
          corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
          corr_invs_sel_supp[ix] = temp
          temp = corr_invs_sel_neus[iy]
          corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
          corr_invs_sel_neus[ix] = temp
          temp = corr_invs_sel_sig[iy]
          corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
          corr_invs_sel_sig[ix] = temp
          temp = corr_invs_sel_labs[iy]
          corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
          corr_invs_sel_labs[ix] = temp
          temp = corr_invs_sel_lay[iy]
          corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
          corr_invs_sel_lay[ix] = temp


for indx in range (0, len(corr_invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(corr_invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(corr_invs_sel_neus[indx])


for indx in range (0, len(corr_invs_sel_sig)):
      Cneuron_sig.append(corr_invs_sel_sig[indx])
      Cneuron_cl.append(corr_invs_sel_labs[indx])
      Cneuron_lay.append(corr_invs_sel_lay[indx])

print("\n CORRECT PATTERNS\n")
for indx in range (0, len(Cneuron_cl)):
  print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


for ix in range (0, len(Minvs_sel_supp)-1):
    for iy in range(ix +1, len(Minvs_sel_supp)):
        if (Minvs_sel_supp[iy] > Minvs_sel_supp[ix]):
          temp = Minvs_sel_supp[iy]
          Minvs_sel_supp[iy] = Minvs_sel_supp[ix]
          Minvs_sel_supp[ix] = temp
          temp = Minvs_sel_neus[iy]
          Minvs_sel_neus[iy] = Minvs_sel_neus[ix]
          Minvs_sel_neus[ix] = temp
          temp = Minvs_sel_sig[iy]
          Minvs_sel_sig[iy] = Minvs_sel_sig[ix]
          Minvs_sel_sig[ix] = temp
          temp = Minvs_sel_labs[iy]
          Minvs_sel_labs[iy] = Minvs_sel_labs[ix]
          Minvs_sel_labs[ix] = temp
          temp = Minvs_sel_lay[iy]
          Minvs_sel_lay[iy] = Minvs_sel_lay[ix]
          Minvs_sel_lay[ix] = temp


for indx in range (0, len(Minvs_sel_neus)):
      MCneu_ids = []
      for indx1 in range (0, len(Minvs_sel_neus[indx])):
        MCneu_ids.append(-1)
        MCneu_ids.append(Minvs_sel_neus[indx][indx1])
      MCneuron_ids.append(MCneu_ids)
      MC_neuron_ids.append(Minvs_sel_neus[indx])


for indx in range (0, len(Minvs_sel_sig)):
      MCneuron_sig.append(Minvs_sel_sig[indx])
      MCneuron_cl.append(Minvs_sel_labs[indx])
      MCneuron_lay.append(Minvs_sel_lay[indx])

print("\n MIS-CLASSIFICATION PATTERNS\n")
for indx in range (0, len(MCneuron_cl)):
  print(MCneuron_cl[indx],Minvs_sel_supp[indx],MCneuron_lay[indx],MC_neuron_ids[indx],MCneuron_sig[indx])

x = copy.deepcopy(val_X)
cnt = 0
correct = 0

labels = (model.predict(x)).argmax(axis=1)
ideals = val_Y




true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
cnt_not_fnd = 0
broke = 0
pois_pat = 0


PASS = 0
PASS_bef = 0
FAIL = 0
FAIL_bef = 0

clean_pass = 0
clean_fail = 0
pois_pass = 0
pois_fail = 0


for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    cnt = cnt + 1
    image = x[inp_cnt]
    label = labels[inp_cnt]

   # inp = image
   # inc_layer = 'dense'  #layer name, get it from the model summary
   # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

    #layer_vals = inc_suff[0]
    #suff = []


    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy][0]))

    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=img
    img_mod = np.expand_dims(img,axis=0)




    inp = image
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []

    inp = image
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]


    inp = image
    inc_layer = 'dense_2'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals3 = inc_suff[0]
    suff = []



    max_lab7 = False
    match1 = -1
    Same = True
    match1 = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,MCneuron_lay,MCneuron_ids,MCneuron_sig)
    print("MATCH MIS-CLASS:", match1)
    if (len(match1) == 0):
      cnt_not_fnd = cnt_not_fnd + 1
      new_lbl = labels[inp_cnt]
    else:
      match = -1
      match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
      if (len(match) == 0):
       # cnt_not_fnd = cnt_not_fnd + 1
        new_lbl = labels[inp_cnt]
      else:
        if (Cneuron_lay[match[0]] == 3):
          if (len(match) > 1):
            new_lbl = Cneuron_cl[match[1]]
          else:
            new_lbl = labels[inp_cnt]
        else:
          new_lbl = Cneuron_cl[match[0]]

        Same = True
        prev_lab = -1
        for indx in range(0,len(match)):
          print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
          if (Cneuron_lay[match[indx]] ==3):
            continue
          if (prev_lab != -1 and Cneuron_cl[match[indx]] != prev_lab):
            Same=False
          prev_lab = Cneuron_cl[match[indx]]




    if (new_lbl == ideals[inp_cnt]):
      PASS = PASS + 1
      if (labels[inp_cnt] != ideals[inp_cnt]):
        correct = correct + 1
        if (max_lab7 == True):
          pois_pat = pois_pat + 1
    else:
      if (Same == False):
        print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
      FAIL = FAIL + 1
      if (labels[inp_cnt] == ideals[inp_cnt]):
        broke = broke + 1

print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
print('CORRECTED:', float(correct/cnt)*100)
print('BROKEN:', float(broke/cnt)*100)
print('POIS PAT',pois_pat)
print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

strat1b_time = (time.time() - strat1b_start_time)/ len(x)




Streaming output truncated to the last 5000 lines.
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: [31]
LAYER,LABEL,SUPP 2 2 319
LAYER,LABEL,SUPP 1 2 14
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: [5]
LAYER,LABEL,SUPP 2 2 20
LAYER,LABEL,SUPP 1 2 20
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS

In [ ]:
#### STRATEGY 1C, CLEAN DATA
strat1c_start_time = time.time()

neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

Cneuron_ids =[]
C_neuron_ids=[]
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []

corr_invs_sel_supp = []
corr_invs_sel_neus = []
corr_invs_sel_sig = []
corr_invs_sel_labs = []
corr_invs_sel_lay = []


MCneuron_ids =[]
MC_neuron_ids=[]
MCneuron_sig = []
MCneuron_cl = []
MCneuron_lay = []

Minvs_sel_supp = []
Minvs_sel_neus = []
Minvs_sel_sig = []
Minvs_sel_labs = []
Minvs_sel_lay = []

for ix in range (0, len(Rcorr_invs_sel_supp1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  corr_invs_sel_supp.append(Rcorr_invs_sel_supp1[ix])
  corr_invs_sel_neus.append(Rcorr_invs_sel_neus1[ix])
  corr_invs_sel_sig.append(Rcorr_invs_sel_sig1[ix])
  corr_invs_sel_labs.append(Rcorr_invs_sel_labs1[ix])
  corr_invs_sel_lay.append(Rcorr_invs_sel_lay1[ix])



for ix in range (0, len(Rcorr_invs_sel_supp2)):
  corr_invs_sel_supp.append(Rcorr_invs_sel_supp2[ix])
  corr_invs_sel_neus.append(Rcorr_invs_sel_neus2[ix])
  corr_invs_sel_sig.append(Rcorr_invs_sel_sig2[ix])
  corr_invs_sel_labs.append(Rcorr_invs_sel_labs2[ix])
  corr_invs_sel_lay.append(Rcorr_invs_sel_lay2[ix])


for ix in range (0, len(PMinvs_sel_sup1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  Minvs_sel_supp.append(PMinvs_sel_sup1[ix])
  Minvs_sel_neus.append(PMinvs_sel_neus1[ix])
  Minvs_sel_sig.append(PMinvs_sel_sig1[ix])
  Minvs_sel_labs.append(PMinvs_sel_cl1[ix])
  Minvs_sel_lay.append(PMinvs_sel_lay1[ix])



for ix in range (0, len(PMinvs_sel_sup2)):
  Minvs_sel_supp.append(PMinvs_sel_sup2[ix])
  Minvs_sel_neus.append(PMinvs_sel_neus2[ix])
  Minvs_sel_sig.append(PMinvs_sel_sig2[ix])
  Minvs_sel_labs.append(PMinvs_sel_cl2[ix])
  Minvs_sel_lay.append(PMinvs_sel_lay2[ix])




for ix in range (0, len(corr_invs_sel_supp)-1):
    for iy in range(ix +1, len(corr_invs_sel_supp)):
        if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
          temp = corr_invs_sel_supp[iy]
          corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
          corr_invs_sel_supp[ix] = temp
          temp = corr_invs_sel_neus[iy]
          corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
          corr_invs_sel_neus[ix] = temp
          temp = corr_invs_sel_sig[iy]
          corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
          corr_invs_sel_sig[ix] = temp
          temp = corr_invs_sel_labs[iy]
          corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
          corr_invs_sel_labs[ix] = temp
          temp = corr_invs_sel_lay[iy]
          corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
          corr_invs_sel_lay[ix] = temp


for indx in range (0, len(corr_invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(corr_invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(corr_invs_sel_neus[indx])


for indx in range (0, len(corr_invs_sel_sig)):
      Cneuron_sig.append(corr_invs_sel_sig[indx])
      Cneuron_cl.append(corr_invs_sel_labs[indx])
      Cneuron_lay.append(corr_invs_sel_lay[indx])

print("\n CORRECT PATTERNS\n")
for indx in range (0, len(Cneuron_cl)):
  print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


for ix in range (0, len(Minvs_sel_supp)-1):
    for iy in range(ix +1, len(Minvs_sel_supp)):
        if (Minvs_sel_supp[iy] > Minvs_sel_supp[ix]):
          temp = Minvs_sel_supp[iy]
          Minvs_sel_supp[iy] = Minvs_sel_supp[ix]
          Minvs_sel_supp[ix] = temp
          temp = Minvs_sel_neus[iy]
          Minvs_sel_neus[iy] = Minvs_sel_neus[ix]
          Minvs_sel_neus[ix] = temp
          temp = Minvs_sel_sig[iy]
          Minvs_sel_sig[iy] = Minvs_sel_sig[ix]
          Minvs_sel_sig[ix] = temp
          temp = Minvs_sel_labs[iy]
          Minvs_sel_labs[iy] = Minvs_sel_labs[ix]
          Minvs_sel_labs[ix] = temp
          temp = Minvs_sel_lay[iy]
          Minvs_sel_lay[iy] = Minvs_sel_lay[ix]
          Minvs_sel_lay[ix] = temp


for indx in range (0, len(Minvs_sel_neus)):
      MCneu_ids = []
      for indx1 in range (0, len(Minvs_sel_neus[indx])):
        MCneu_ids.append(-1)
        MCneu_ids.append(Minvs_sel_neus[indx][indx1])
      MCneuron_ids.append(MCneu_ids)
      MC_neuron_ids.append(Minvs_sel_neus[indx])


for indx in range (0, len(Minvs_sel_sig)):
      MCneuron_sig.append(Minvs_sel_sig[indx])
      MCneuron_cl.append(Minvs_sel_labs[indx])
      MCneuron_lay.append(Minvs_sel_lay[indx])

print("\n MIS-CLASSIFICATION PATTERNS\n")
for indx in range (0, len(MCneuron_cl)):
  print(MCneuron_cl[indx],Minvs_sel_supp[indx],MCneuron_lay[indx],MC_neuron_ids[indx],MCneuron_sig[indx])

x = copy.deepcopy(val_X)
cnt = 0
correct = 0

labels = (model.predict(x)).argmax(axis=1)
ideals = val_Y




true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
cnt_not_fnd = 0
broke = 0
pois_pat = 0


PASS = 0
PASS_bef = 0
FAIL = 0
FAIL_bef = 0

clean_pass = 0
clean_fail = 0
pois_pass = 0
pois_fail = 0


for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    cnt = cnt + 1
    image = x[inp_cnt]
    label = labels[inp_cnt]

   # inp = image
   # inc_layer = 'dense'  #layer name, get it from the model summary
   # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

    #layer_vals = inc_suff[0]
    #suff = []


    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy][0]))

    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=img
    img_mod = np.expand_dims(img,axis=0)


    inp = image
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []

    inp = image
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]

    inp = image
    inc_layer = 'dense_2'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals3 = inc_suff[0]
    suff = []



    max_lab7 = False
    match1 = -1

    Same = True
    match1 = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,MCneuron_lay,MCneuron_ids,MCneuron_sig)
    print("MATCH MIS-CLASS:", match1)
    if (len(match1) == 0):
      cnt_not_fnd = cnt_not_fnd + 1
      new_lbl = labels[inp_cnt]
    else:
      match = -1
      match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
      if (len(match) == 0):
      #  cnt_not_fnd = cnt_not_fnd + 1
        new_lbl = labels[inp_cnt]
      else:
        if (Cneuron_lay[match[0]] == 3):
          if (len(match) > 1):
            new_lbl = Cneuron_cl[match[1]]
          else:
            new_lbl = labels[inp_cnt]
        else:
          new_lbl = Cneuron_cl[match[0]]

        Same = True
        prev_lab = -1
        for indx in range(0,len(match)):
          print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
          if (Cneuron_lay[match[indx]] ==3):
            continue
          if (prev_lab != -1 and Cneuron_cl[match[indx]] != prev_lab):
            Same=False
          prev_lab = Cneuron_cl[match[indx]]




    if (new_lbl == ideals[inp_cnt]):
      PASS = PASS + 1
      if (labels[inp_cnt] != ideals[inp_cnt]):
        correct = correct + 1
        if (max_lab7 == True):
          pois_pat = pois_pat + 1
    else:
      if (Same == False):
        print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
      FAIL = FAIL + 1
      if (labels[inp_cnt] == ideals[inp_cnt]):
        broke = broke + 1

print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
print('CORRECTED:', float(correct/cnt)*100)
print('BROKEN:', float(broke/cnt)*100)
print('POIS PAT',pois_pat)
print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

strat1c_time = (time.time() - strat1c_start_time)/ len(x)



Streaming output truncated to the last 5000 lines.
MATCH MIS-CLASS: [31]
LAYER,LABEL,SUPP 2 2 93
LAYER,LABEL,SUPP 1 2 10
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: [5]
LAYER,LABEL,SUPP 2 7 394
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLASS: []
MATCH MIS-CLA

In [ ]:
#### STRATEGY 1A, POIS DATA
strat1a_start_pois_time = time.time()

neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

Cneuron_ids =[]
C_neuron_ids=[]
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []

corr_invs_sel_supp = []
corr_invs_sel_neus = []
corr_invs_sel_sig = []
corr_invs_sel_labs = []
corr_invs_sel_lay = []

for ix in range (0, len(corr_invs_sel_supp1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
  corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
  corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
  corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
  corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])

# for ix in range (0, len(corr_invs_sel_supp1a)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp1a[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus1a[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig1a[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs1a[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay1a[ix])

for ix in range (0, len(corr_invs_sel_supp2)):
  corr_invs_sel_supp.append(corr_invs_sel_supp2[ix])
  corr_invs_sel_neus.append(corr_invs_sel_neus2[ix])
  corr_invs_sel_sig.append(corr_invs_sel_sig2[ix])
  corr_invs_sel_labs.append(corr_invs_sel_labs2[ix])
  corr_invs_sel_lay.append(corr_invs_sel_lay2[ix])

# for ix in range (0, len(corr_invs_sel_supp2a)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp2a[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus2a[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig2a[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs2a[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay2a[ix])




for ix in range (0, len(corr_invs_sel_supp)-1):
    for iy in range(ix +1, len(corr_invs_sel_supp)):
        if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
          temp = corr_invs_sel_supp[iy]
          corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
          corr_invs_sel_supp[ix] = temp
          temp = corr_invs_sel_neus[iy]
          corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
          corr_invs_sel_neus[ix] = temp
          temp = corr_invs_sel_sig[iy]
          corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
          corr_invs_sel_sig[ix] = temp
          temp = corr_invs_sel_labs[iy]
          corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
          corr_invs_sel_labs[ix] = temp
          temp = corr_invs_sel_lay[iy]
          corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
          corr_invs_sel_lay[ix] = temp


for indx in range (0, len(corr_invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(corr_invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(corr_invs_sel_neus[indx])


for indx in range (0, len(corr_invs_sel_sig)):
      Cneuron_sig.append(corr_invs_sel_sig[indx])
      Cneuron_cl.append(corr_invs_sel_labs[indx])
      Cneuron_lay.append(corr_invs_sel_lay[indx])

print("\n CORRECT PATTERNS\n")
for indx in range (0, len(Cneuron_cl)):
  print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


x = copy.deepcopy(Pval_X)
cnt = 0
correct = 0

labels = (model.predict(x)).argmax(axis=1)
ideals = Pval_Y




true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
cnt_not_fnd = 0
broke = 0
pois_pat = 0


PASS = 0
PASS_bef = 0
FAIL = 0
FAIL_bef = 0

clean_pass = 0
clean_fail = 0
pois_pass = 0
pois_fail = 0


for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    cnt = cnt + 1
    image = x[inp_cnt]
    label = labels[inp_cnt]

   # inp = image
   # inc_layer = 'dense'  #layer name, get it from the model summary
   # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

    #layer_vals = inc_suff[0]
    #suff = []


    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy][0]))

    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=img
    img_mod = np.expand_dims(img,axis=0)


    inp = image
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []

    inp = image
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]

    inp = image
    inc_layer = 'dense_2'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals3 = inc_suff[0]
    suff = []



    max_lab7 = False
    match = -1
    Same = True
    match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
    print("MATCH:", match)
    if (len(match) == 0):
      cnt_not_fnd = cnt_not_fnd + 1
      new_lbl = labels[inp_cnt]
    else:
      if (Cneuron_lay[match[0]] == 3):
        max_lab7 = True
        if (len(match) > 1):
          new_lbl = Cneuron_cl[match[1]]
        else:
          new_lbl = labels[inp_cnt]
      else:
        new_lbl = Cneuron_cl[match[0]]

      Same = True
      prev_lab = -1
      for indx in range(0,len(match)):
         print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
         if (Cneuron_lay[match[indx]] ==3):
           continue
         if (prev_lab != -1 and Cneuron_cl[match[indx]] != prev_lab):
             Same=False
         prev_lab = Cneuron_cl[match[indx]]




    if (new_lbl == ideals[inp_cnt]):
      PASS = PASS + 1
      if (labels[inp_cnt] != ideals[inp_cnt]):
        correct = correct + 1
        if (max_lab7 == True):
          pois_pat = pois_pat + 1
    else:
      if (Same == False):
        print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
      FAIL = FAIL + 1
      if (labels[inp_cnt] == ideals[inp_cnt]):
        broke = broke + 1

print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
print('CORRECTED:', float(correct/cnt)*100)
print('BROKEN:', float(broke/cnt)*100)
print('POIS PAT',pois_pat)
print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

strat1a_pois_time = (time.time() - strat1a_start_pois_time)/ len(x)

print("STRAT 1A RUNTIME:", (strat1a_pois_time + strat1a_time)/2.0)


Streaming output truncated to the last 5000 lines.
LAYER,LABEL,SUPP 2 7 10
MATCH: [13, 15]
LAYER,LABEL,SUPP 2 7 319
LAYER,LABEL,SUPP 1 7 297
MATCH: [131, 133]
LAYER,LABEL,SUPP 2 7 5
LAYER,LABEL,SUPP 1 7 5
MATCH: [46, 48]
LAYER,LABEL,SUPP 2 2 20
LAYER,LABEL,SUPP 1 2 20
MATCH: [47]
LAYER,LABEL,SUPP 2 8 20
MATCH: [46, 48]
LAYER,LABEL,SUPP 2 2 20
LAYER,LABEL,SUPP 1 2 20
MATCH: [13, 451]
LAYER,LABEL,SUPP 2 7 319
LAYER,LABEL,SUPP 1 5 1
OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL: 7 5 [13, 451] 7
MATCH: [2, 3]
LAYER,LABEL,SUPP 2 0 436
LAYER,LABEL,SUPP 1 0 436
MATCH: [13, 15]
LAYER,LABEL,SUPP 2 7 319
LAYER,LABEL,SUPP 1 7 297
MATCH: [13, 15]
LAYER,LABEL,SUPP 2 7 319
LAYER,LABEL,SUPP 1 7 297
MATCH: [0, 210]
LAYER,LABEL,SUPP 1 1 506
LAYER,LABEL,SUPP 2 7 3
MATCH: [0, 290]
LAYER,LABEL,SUPP 1 1 506
LAYER,LABEL,SUPP 2 8 2
MATCH: [46, 133]
LAYER,LABEL,SUPP 2 2 20
LAYER,LABEL,SUPP 1 7 5
OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL: 7 6 [46, 133] 2
MATCH: [131, 133]
LAYER,LABEL,SUPP 2 7 5
LAYER,LABEL,SUPP 1 7 5


In [ ]:
#### STRATEGY 1B, POIS DATA
strat1b_start_pois_time = time.time()

neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

Cneuron_ids =[]
C_neuron_ids=[]
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []

corr_invs_sel_supp = []
corr_invs_sel_neus = []
corr_invs_sel_sig = []
corr_invs_sel_labs = []
corr_invs_sel_lay = []


MCneuron_ids =[]
MC_neuron_ids=[]
MCneuron_sig = []
MCneuron_cl = []
MCneuron_lay = []

Minvs_sel_supp = []
Minvs_sel_neus = []
Minvs_sel_sig = []
Minvs_sel_labs = []
Minvs_sel_lay = []

for ix in range (0, len(corr_invs_sel_supp1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
  corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
  corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
  corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
  corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])



for ix in range (0, len(corr_invs_sel_supp2)):
  corr_invs_sel_supp.append(corr_invs_sel_supp2[ix])
  corr_invs_sel_neus.append(corr_invs_sel_neus2[ix])
  corr_invs_sel_sig.append(corr_invs_sel_sig2[ix])
  corr_invs_sel_labs.append(corr_invs_sel_labs2[ix])
  corr_invs_sel_lay.append(corr_invs_sel_lay2[ix])


for ix in range (0, len(PMinvs_sel_sup1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  Minvs_sel_supp.append(PMinvs_sel_sup1[ix])
  Minvs_sel_neus.append(PMinvs_sel_neus1[ix])
  Minvs_sel_sig.append(PMinvs_sel_sig1[ix])
  Minvs_sel_labs.append(PMinvs_sel_cl1[ix])
  Minvs_sel_lay.append(PMinvs_sel_lay1[ix])



for ix in range (0, len(PMinvs_sel_sup2)):
  Minvs_sel_supp.append(PMinvs_sel_sup2[ix])
  Minvs_sel_neus.append(PMinvs_sel_neus2[ix])
  Minvs_sel_sig.append(PMinvs_sel_sig2[ix])
  Minvs_sel_labs.append(PMinvs_sel_cl2[ix])
  Minvs_sel_lay.append(PMinvs_sel_lay2[ix])




for ix in range (0, len(corr_invs_sel_supp)-1):
    for iy in range(ix +1, len(corr_invs_sel_supp)):
        if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
          temp = corr_invs_sel_supp[iy]
          corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
          corr_invs_sel_supp[ix] = temp
          temp = corr_invs_sel_neus[iy]
          corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
          corr_invs_sel_neus[ix] = temp
          temp = corr_invs_sel_sig[iy]
          corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
          corr_invs_sel_sig[ix] = temp
          temp = corr_invs_sel_labs[iy]
          corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
          corr_invs_sel_labs[ix] = temp
          temp = corr_invs_sel_lay[iy]
          corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
          corr_invs_sel_lay[ix] = temp


for indx in range (0, len(corr_invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(corr_invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(corr_invs_sel_neus[indx])


for indx in range (0, len(corr_invs_sel_sig)):
      Cneuron_sig.append(corr_invs_sel_sig[indx])
      Cneuron_cl.append(corr_invs_sel_labs[indx])
      Cneuron_lay.append(corr_invs_sel_lay[indx])

print("\n CORRECT PATTERNS\n")
for indx in range (0, len(Cneuron_cl)):
  print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


for ix in range (0, len(Minvs_sel_supp)-1):
    for iy in range(ix +1, len(Minvs_sel_supp)):
        if (Minvs_sel_supp[iy] > Minvs_sel_supp[ix]):
          temp = Minvs_sel_supp[iy]
          Minvs_sel_supp[iy] = Minvs_sel_supp[ix]
          Minvs_sel_supp[ix] = temp
          temp = Minvs_sel_neus[iy]
          Minvs_sel_neus[iy] = Minvs_sel_neus[ix]
          Minvs_sel_neus[ix] = temp
          temp = Minvs_sel_sig[iy]
          Minvs_sel_sig[iy] = Minvs_sel_sig[ix]
          Minvs_sel_sig[ix] = temp
          temp = Minvs_sel_labs[iy]
          Minvs_sel_labs[iy] = Minvs_sel_labs[ix]
          Minvs_sel_labs[ix] = temp
          temp = Minvs_sel_lay[iy]
          Minvs_sel_lay[iy] = Minvs_sel_lay[ix]
          Minvs_sel_lay[ix] = temp


for indx in range (0, len(Minvs_sel_neus)):
      MCneu_ids = []
      for indx1 in range (0, len(Minvs_sel_neus[indx])):
        MCneu_ids.append(-1)
        MCneu_ids.append(Minvs_sel_neus[indx][indx1])
      MCneuron_ids.append(MCneu_ids)
      MC_neuron_ids.append(Minvs_sel_neus[indx])


for indx in range (0, len(Minvs_sel_sig)):
      MCneuron_sig.append(Minvs_sel_sig[indx])
      MCneuron_cl.append(Minvs_sel_labs[indx])
      MCneuron_lay.append(Minvs_sel_lay[indx])

print("\n MIS-CLASSIFICATION PATTERNS\n")
for indx in range (0, len(MCneuron_cl)):
  print(MCneuron_cl[indx],Minvs_sel_supp[indx],MCneuron_lay[indx],MC_neuron_ids[indx],MCneuron_sig[indx])

x = copy.deepcopy(Pval_X)
cnt = 0
correct = 0

labels = (model.predict(x)).argmax(axis=1)
ideals = Pval_Y




true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
cnt_not_fnd = 0
broke = 0
pois_pat = 0


PASS = 0
PASS_bef = 0
FAIL = 0
FAIL_bef = 0

clean_pass = 0
clean_fail = 0
pois_pass = 0
pois_fail = 0


for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    cnt = cnt + 1
    image = x[inp_cnt]
    label = labels[inp_cnt]

   # inp = image
   # inc_layer = 'dense'  #layer name, get it from the model summary
   # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

    #layer_vals = inc_suff[0]
    #suff = []



    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy][0]))

    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=img
    img_mod = np.expand_dims(img,axis=0)

    inp = image
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []

    inp = image
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]

    inp = image
    inc_layer = 'dense_2'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals3 = inc_suff[0]
    suff = []



    max_lab7 = False
    match1 = -1
    Same = True
    match1 = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,MCneuron_lay,MCneuron_ids,MCneuron_sig)
    print("MATCH MIS-CLASS:", match1)
    if (len(match1) == 0):
      cnt_not_fnd = cnt_not_fnd + 1
      new_lbl = labels[inp_cnt]
    else:
      match = -1
      match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
      if (len(match) == 0):
      #  cnt_not_fnd = cnt_not_fnd + 1
        new_lbl = labels[inp_cnt]
      else:
        if (Cneuron_lay[match[0]] == 3):
          if (len(match) > 1):
            new_lbl = Cneuron_cl[match[1]]
          else:
            new_lbl = labels[inp_cnt]
        else:
          new_lbl = Cneuron_cl[match[0]]

        Same = True
        prev_lab = -1
        for indx in range(0,len(match)):
          print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
          if (Cneuron_lay[match[indx]] ==3):
            continue
          if (prev_lab != -1 and Cneuron_cl[match[indx]] != prev_lab):
            Same=False
          prev_lab = Cneuron_cl[match[indx]]




    if (new_lbl == ideals[inp_cnt]):
      PASS = PASS + 1
      if (labels[inp_cnt] != ideals[inp_cnt]):
        correct = correct + 1
        if (max_lab7 == True):
          pois_pat = pois_pat + 1
    else:
      if (Same == False):
        print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
      FAIL = FAIL + 1
      if (labels[inp_cnt] == ideals[inp_cnt]):
        broke = broke + 1

print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
print('CORRECTED:', float(correct/cnt)*100)
print('BROKEN:', float(broke/cnt)*100)
print('POIS PAT',pois_pat)
print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

strat1b_pois_time = (time.time() - strat1b_start_pois_time)/ len(x)

print("STRAT 1B RUNTIME:", (strat1b_pois_time + strat1b_time)/2.0)



Streaming output truncated to the last 5000 lines.
LAYER,LABEL,SUPP 2 7 5
OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL: 7 3 [48, 131] 2
MATCH MIS-CLASS: [3, 6]
LAYER,LABEL,SUPP 2 2 319
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 2 7 5
LAYER,LABEL,SUPP 1 7 5
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 2 7 319
LAYER,LABEL,SUPP 1 7 297
MATCH MIS-CLASS: [7, 8]
LAYER,LABEL,SUPP 2 7 93
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 2 2 20
LAYER,LABEL,SUPP 1 2 20
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 2 7 5
LAYER,LABEL,SUPP 1 7 5
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 2 7 5
LAYER,LABEL,SUPP 1 7 5
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 2 7 319
LAYER,LABEL,SUPP 1 7 297
MATCH MIS-CLASS: [14]
LAYER,LABEL,SUPP 2 7 5
LAYER,LABEL,SUPP 1 7 5
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 1 2 20
LAYER,LABEL,SUPP 2 7 5
MATCH MIS-CLASS: []
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 2 5 222
LAYER,LABEL,SUPP 1 5 222
MATCH MIS-CLASS: [8]
LAYER,LABEL,SUPP 2 7 5
LAYER,LABEL,SUPP 1 7 5
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 2

In [ ]:
#### STRATEGY 1C, POIS DATA
strat1c_start_pois_time = time.time()

neuron_ids =[]
inc_neuron_ids=[]
neuron_sig = []

Cneuron_ids =[]
C_neuron_ids=[]
Cneuron_sig = []
Cneuron_cl = []
Cneuron_lay = []

corr_invs_sel_supp = []
corr_invs_sel_neus = []
corr_invs_sel_sig = []
corr_invs_sel_labs = []
corr_invs_sel_lay = []


MCneuron_ids =[]
MC_neuron_ids=[]
MCneuron_sig = []
MCneuron_cl = []
MCneuron_lay = []

Minvs_sel_supp = []
Minvs_sel_neus = []
Minvs_sel_sig = []
Minvs_sel_labs = []
Minvs_sel_lay = []

for ix in range (0, len(Rcorr_invs_sel_supp1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  corr_invs_sel_supp.append(Rcorr_invs_sel_supp1[ix])
  corr_invs_sel_neus.append(Rcorr_invs_sel_neus1[ix])
  corr_invs_sel_sig.append(Rcorr_invs_sel_sig1[ix])
  corr_invs_sel_labs.append(Rcorr_invs_sel_labs1[ix])
  corr_invs_sel_lay.append(Rcorr_invs_sel_lay1[ix])



for ix in range (0, len(Rcorr_invs_sel_supp2)):
  corr_invs_sel_supp.append(Rcorr_invs_sel_supp2[ix])
  corr_invs_sel_neus.append(Rcorr_invs_sel_neus2[ix])
  corr_invs_sel_sig.append(Rcorr_invs_sel_sig2[ix])
  corr_invs_sel_labs.append(Rcorr_invs_sel_labs2[ix])
  corr_invs_sel_lay.append(Rcorr_invs_sel_lay2[ix])


for ix in range (0, len(PMinvs_sel_sup1)):
  #if (corr_invs_sel_supp1[ix] <= 20):
  #   continue
  Minvs_sel_supp.append(PMinvs_sel_sup1[ix])
  Minvs_sel_neus.append(PMinvs_sel_neus1[ix])
  Minvs_sel_sig.append(PMinvs_sel_sig1[ix])
  Minvs_sel_labs.append(PMinvs_sel_cl1[ix])
  Minvs_sel_lay.append(PMinvs_sel_lay1[ix])



for ix in range (0, len(PMinvs_sel_sup2)):
  Minvs_sel_supp.append(PMinvs_sel_sup2[ix])
  Minvs_sel_neus.append(PMinvs_sel_neus2[ix])
  Minvs_sel_sig.append(PMinvs_sel_sig2[ix])
  Minvs_sel_labs.append(PMinvs_sel_cl2[ix])
  Minvs_sel_lay.append(PMinvs_sel_lay2[ix])




for ix in range (0, len(corr_invs_sel_supp)-1):
    for iy in range(ix +1, len(corr_invs_sel_supp)):
        if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
          temp = corr_invs_sel_supp[iy]
          corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
          corr_invs_sel_supp[ix] = temp
          temp = corr_invs_sel_neus[iy]
          corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
          corr_invs_sel_neus[ix] = temp
          temp = corr_invs_sel_sig[iy]
          corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
          corr_invs_sel_sig[ix] = temp
          temp = corr_invs_sel_labs[iy]
          corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
          corr_invs_sel_labs[ix] = temp
          temp = corr_invs_sel_lay[iy]
          corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
          corr_invs_sel_lay[ix] = temp


for indx in range (0, len(corr_invs_sel_neus)):
      Cneu_ids = []
      for indx1 in range (0, len(corr_invs_sel_neus[indx])):
        Cneu_ids.append(-1)
        Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
      Cneuron_ids.append(Cneu_ids)
      C_neuron_ids.append(corr_invs_sel_neus[indx])


for indx in range (0, len(corr_invs_sel_sig)):
      Cneuron_sig.append(corr_invs_sel_sig[indx])
      Cneuron_cl.append(corr_invs_sel_labs[indx])
      Cneuron_lay.append(corr_invs_sel_lay[indx])

print("\n CORRECT PATTERNS\n")
for indx in range (0, len(Cneuron_cl)):
  print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


for ix in range (0, len(Minvs_sel_supp)-1):
    for iy in range(ix +1, len(Minvs_sel_supp)):
        if (Minvs_sel_supp[iy] > Minvs_sel_supp[ix]):
          temp = Minvs_sel_supp[iy]
          Minvs_sel_supp[iy] = Minvs_sel_supp[ix]
          Minvs_sel_supp[ix] = temp
          temp = Minvs_sel_neus[iy]
          Minvs_sel_neus[iy] = Minvs_sel_neus[ix]
          Minvs_sel_neus[ix] = temp
          temp = Minvs_sel_sig[iy]
          Minvs_sel_sig[iy] = Minvs_sel_sig[ix]
          Minvs_sel_sig[ix] = temp
          temp = Minvs_sel_labs[iy]
          Minvs_sel_labs[iy] = Minvs_sel_labs[ix]
          Minvs_sel_labs[ix] = temp
          temp = Minvs_sel_lay[iy]
          Minvs_sel_lay[iy] = Minvs_sel_lay[ix]
          Minvs_sel_lay[ix] = temp


for indx in range (0, len(Minvs_sel_neus)):
      MCneu_ids = []
      for indx1 in range (0, len(Minvs_sel_neus[indx])):
        MCneu_ids.append(-1)
        MCneu_ids.append(Minvs_sel_neus[indx][indx1])
      MCneuron_ids.append(MCneu_ids)
      MC_neuron_ids.append(Minvs_sel_neus[indx])


for indx in range (0, len(Minvs_sel_sig)):
      MCneuron_sig.append(Minvs_sel_sig[indx])
      MCneuron_cl.append(Minvs_sel_labs[indx])
      MCneuron_lay.append(Minvs_sel_lay[indx])

print("\n MIS-CLASSIFICATION PATTERNS\n")
for indx in range (0, len(MCneuron_cl)):
  print(MCneuron_cl[indx],Minvs_sel_supp[indx],MCneuron_lay[indx],MC_neuron_ids[indx],MCneuron_sig[indx])

x = copy.deepcopy(Pval_X)
cnt = 0
correct = 0

labels = (model.predict(x)).argmax(axis=1)
ideals = Pval_Y




true_positives = 0
true_negatives = 0
false_positives = 0
false_negatives = 0
cnt_not_fnd = 0
broke = 0
pois_pat = 0


PASS = 0
PASS_bef = 0
FAIL = 0
FAIL_bef = 0

clean_pass = 0
clean_fail = 0
pois_pass = 0
pois_fail = 0


for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
    cnt = cnt + 1
    image = x[inp_cnt]
    label = labels[inp_cnt]

   # inp = image
   # inc_layer = 'dense'  #layer name, get it from the model summary
   # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

    #layer_vals = inc_suff[0]
    #suff = []


    img = np.zeros((28,28),dtype=float)
    for ix in range(0,28):
      for iy in range(0,28):
          img[ix][iy] = (float(image[ix][iy][0]))

    image_array = np.zeros((1,28,28),dtype=float)
    image_array[0]=img
    img_mod = np.expand_dims(img,axis=0)


    inp = image
    inc_layer = 'dense_1'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals1 = inc_suff[0]
    suff = []

    inp = image
    inc_layer = 'activation_3'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals2 = inc_suff[0]

    inp = image
    inc_layer = 'dense_2'  #layer name, get it from the model summary
    inc_suff = fingerprint_suffix_vals(inp, inc_layer)
    layer_vals3 = inc_suff[0]
    suff = []



    max_lab7 = False
    match1 = -1
    Same = True
    match1 = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,MCneuron_lay,MCneuron_ids,MCneuron_sig)
    print("MATCH MIS-CLASS:", match1)
    if (len(match1) == 0):
      cnt_not_fnd = cnt_not_fnd + 1
      new_lbl = labels[inp_cnt]
    else:
      match = -1
      match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
      if (len(match) == 0):
      #  cnt_not_fnd = cnt_not_fnd + 1
        new_lbl = labels[inp_cnt]
      else:
        if (Cneuron_lay[match[0]] == 3):
          if (len(match) > 1):
            new_lbl = Cneuron_cl[match[1]]
          else:
            new_lbl = labels[inp_cnt]
        else:
          new_lbl = Cneuron_cl[match[0]]

        Same = True
        prev_lab = -1
        for indx in range(0,len(match)):
          print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
          if (Cneuron_lay[match[indx]] ==3):
            continue
          if (prev_lab != -1 and Cneuron_cl[match[indx]] != prev_lab):
            Same=False
          prev_lab = Cneuron_cl[match[indx]]




    if (new_lbl == ideals[inp_cnt]):
      PASS = PASS + 1
      if (labels[inp_cnt] != ideals[inp_cnt]):
        correct = correct + 1
        if (max_lab7 == True):
          pois_pat = pois_pat + 1
    else:
      if (Same == False):
        print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
      FAIL = FAIL + 1
      if (labels[inp_cnt] == ideals[inp_cnt]):
        broke = broke + 1

print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
print('CORRECTED:', float(correct/cnt)*100)
print('BROKEN:', float(broke/cnt)*100)
print('POIS PAT',pois_pat)
print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

strat1c_pois_time = (time.time() - strat1c_start_pois_time)/ len(x)

print("STRAT 1C RUNTIME:", (strat1c_pois_time + strat1c_time)/2.0)


Streaming output truncated to the last 5000 lines.
LAYER,LABEL,SUPP 2 7 4
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 2 3 15
LAYER,LABEL,SUPP 1 6 13
MATCH MIS-CLASS: [3, 6]
LAYER,LABEL,SUPP 2 8 79
LAYER,LABEL,SUPP 1 8 2
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 1 3 50
LAYER,LABEL,SUPP 2 3 15
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 1 9 368
LAYER,LABEL,SUPP 2 9 350
MATCH MIS-CLASS: [7, 8]
LAYER,LABEL,SUPP 1 0 62
LAYER,LABEL,SUPP 2 9 26
OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL: 7 9 [24, 57] 0
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 2 5 1
LAYER,LABEL,SUPP 1 2 1
OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL: 7 4 [518, 669] 5
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 1 7 431
LAYER,LABEL,SUPP 2 3 15
OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL: 7 8 [8, 94] 7
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 2 2 44
LAYER,LABEL,SUPP 1 7 4
OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL: 7 3 [36, 226] 2
MATCH MIS-CLASS: [0, 1]
LAYER,LABEL,SUPP 1 9 368
LAYER,LABEL,SUPP 2 4 4
OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL: 7 4 [11, 210

In [ ]:
##### STOP HERE OLD RESULTS

In [ ]:

# #### GUESSING THE IDEAL LABELS BASED ON PATTERNS
# neuron_ids =[]
# inc_neuron_ids=[]
# neuron_sig = []

# Cneuron_ids =[]
# C_neuron_ids=[]
# Cneuron_sig = []
# Cneuron_cl = []
# Cneuron_lay = []

# corr_invs_sel_supp = []
# corr_invs_sel_neus = []
# corr_invs_sel_sig = []
# corr_invs_sel_labs = []
# corr_invs_sel_lay = []

# for ix in range (0, len(corr_invs_sel_supp1)):
#   #if (corr_invs_sel_supp1[ix] <= 20):
#   #   continue
#   corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])

# # for ix in range (0, len(corr_invs_sel_supp1a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp1a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus1a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig1a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs1a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay1a[ix])

# for ix in range (0, len(corr_invs_sel_supp2)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp2[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus2[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig2[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs2[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay2[ix])

# # for ix in range (0, len(corr_invs_sel_supp2a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp2a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus2a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig2a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs2a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay2a[ix])

# for ix in range (0, len(corr_invs_sel_supp3)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp3[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus3[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig3[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs3[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay3[ix])

# # for ix in range (0, len(corr_invs_sel_supp3a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp3a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus3a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig3a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs3a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay3a[ix])


# for ix in range (0, len(corr_invs_sel_supp)-1):
#     for iy in range(ix +1, len(corr_invs_sel_supp)):
#         if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
#           temp = corr_invs_sel_supp[iy]
#           corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
#           corr_invs_sel_supp[ix] = temp
#           temp = corr_invs_sel_neus[iy]
#           corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
#           corr_invs_sel_neus[ix] = temp
#           temp = corr_invs_sel_sig[iy]
#           corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
#           corr_invs_sel_sig[ix] = temp
#           temp = corr_invs_sel_labs[iy]
#           corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
#           corr_invs_sel_labs[ix] = temp
#           temp = corr_invs_sel_lay[iy]
#           corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
#           corr_invs_sel_lay[ix] = temp


# for indx in range (0, len(corr_invs_sel_neus)):
#       Cneu_ids = []
#       for indx1 in range (0, len(corr_invs_sel_neus[indx])):
#         Cneu_ids.append(-1)
#         Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
#       Cneuron_ids.append(Cneu_ids)
#       C_neuron_ids.append(corr_invs_sel_neus[indx])


# for indx in range (0, len(corr_invs_sel_sig)):
#       Cneuron_sig.append(corr_invs_sel_sig[indx])
#       Cneuron_cl.append(corr_invs_sel_labs[indx])
#       Cneuron_lay.append(corr_invs_sel_lay[indx])

# print("\n CORRECT PATTERNS\n")
# for indx in range (0, len(Cneuron_cl)):
#   print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


# x = copy.deepcopy(test_X_pois)
# cnt = 0
# correct = 0

# labels = (model.predict(x)).argmax(axis=1)
# ideals = testFULL_Y






# true_positives = 0
# true_negatives = 0
# false_positives = 0
# false_negatives = 0
# cnt_not_fnd = 0
# broke = 0
# pois_pat = 0


# PASS = 0
# PASS_bef = 0
# FAIL = 0
# FAIL_bef = 0

# clean_pass = 0
# clean_fail = 0
# pois_pass = 0
# pois_fail = 0


# for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
#     cnt = cnt + 1
#     image = x[inp_cnt]
#     label = labels[inp_cnt]

#    # inp = image
#    # inc_layer = 'dense'  #layer name, get it from the model summary
#    # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

#     #layer_vals = inc_suff[0]
#     #suff = []



#     img = np.zeros((28,28),dtype=float)
#     for ix in range(0,28):
#       for iy in range(0,28):
#           img[ix][iy] = (float(image[ix][iy][0]))

#     image_array = np.zeros((1,28,28),dtype=float)
#     image_array[0]=img
#     img_mod = np.expand_dims(img,axis=0)


#     inp = image
#     inc_layer = 'dense_1'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals1 = inc_suff[0]
#     suff = []

#     inp = image
#     inc_layer = 'activation_3'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals2 = inc_suff[0]

#     inp = image
#     inc_layer = 'dense_2'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals3 = inc_suff[0]
#     suff = []


#     max_lab7 = False
#     match = -1
#     match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
#     print("MATCH:", match)
#     if (len(match) == 0):
#       cnt_not_fnd = cnt_not_fnd + 1
#       new_lbl = labels[inp_cnt]
#     else:
#       if (Cneuron_lay[match[0]] == 3):
#         max_lab7 = True
#         if (len(match) > 1):
#           new_lbl = Cneuron_cl[match[1]]
#         else:
#           new_lbl = labels[inp_cnt]
#       else:
#         new_lbl = Cneuron_cl[match[0]]






#     if (new_lbl == ideals[inp_cnt]):
#       PASS = PASS + 1
#       if (labels[inp_cnt] != ideals[inp_cnt]):
#         correct = correct + 1
#         if (max_lab7 == True):
#           pois_pat = pois_pat + 1
#     else:
#       for indx in range(0,len(match)):
#          print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
#       print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
#       FAIL = FAIL + 1
#       if (labels[inp_cnt] == ideals[inp_cnt]):
#         broke = broke + 1
#       #print("BROKE: OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)


# print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

# print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
# print('CORRECTED:', float(correct/cnt)*100)
# print('BROKEN:', float(broke/cnt)*100)
# print('POIS PAT',pois_pat)
# print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )


In [ ]:

# #### GUESSING THE IDEAL LABELS BASED ON PATTERNS
# neuron_ids =[]
# inc_neuron_ids=[]
# neuron_sig = []

# Cneuron_ids =[]
# C_neuron_ids=[]
# Cneuron_sig = []
# Cneuron_cl = []
# Cneuron_lay = []

# corr_invs_sel_supp = []
# corr_invs_sel_neus = []
# corr_invs_sel_sig = []
# corr_invs_sel_labs = []
# corr_invs_sel_lay = []

# for ix in range (0, len(corr_invs_sel_supp1)):
#   #if (corr_invs_sel_supp1[ix] <= 20):
#   #   continue
#   corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])

# # for ix in range (0, len(corr_invs_sel_supp1a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp1a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus1a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig1a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs1a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay1a[ix])

# for ix in range (0, len(corr_invs_sel_supp2)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp2[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus2[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig2[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs2[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay2[ix])

# # for ix in range (0, len(corr_invs_sel_supp2a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp2a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus2a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig2a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs2a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay2a[ix])

# for ix in range (0, len(corr_invs_sel_supp3)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp3[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus3[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig3[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs3[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay3[ix])

# # for ix in range (0, len(corr_invs_sel_supp3a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp3a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus3a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig3a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs3a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay3a[ix])


# for ix in range (0, len(corr_invs_sel_supp)-1):
#     for iy in range(ix +1, len(corr_invs_sel_supp)):
#         if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
#           temp = corr_invs_sel_supp[iy]
#           corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
#           corr_invs_sel_supp[ix] = temp
#           temp = corr_invs_sel_neus[iy]
#           corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
#           corr_invs_sel_neus[ix] = temp
#           temp = corr_invs_sel_sig[iy]
#           corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
#           corr_invs_sel_sig[ix] = temp
#           temp = corr_invs_sel_labs[iy]
#           corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
#           corr_invs_sel_labs[ix] = temp
#           temp = corr_invs_sel_lay[iy]
#           corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
#           corr_invs_sel_lay[ix] = temp


# for indx in range (0, len(corr_invs_sel_neus)):
#       Cneu_ids = []
#       for indx1 in range (0, len(corr_invs_sel_neus[indx])):
#         Cneu_ids.append(-1)
#         Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
#       Cneuron_ids.append(Cneu_ids)
#       C_neuron_ids.append(corr_invs_sel_neus[indx])


# for indx in range (0, len(corr_invs_sel_sig)):
#       Cneuron_sig.append(corr_invs_sel_sig[indx])
#       Cneuron_cl.append(corr_invs_sel_labs[indx])
#       Cneuron_lay.append(corr_invs_sel_lay[indx])

# print("\n CORRECT PATTERNS\n")
# for indx in range (0, len(Cneuron_cl)):
#   print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


# x = copy.deepcopy(test_X_pois)
# cnt = 0
# correct = 0

# labels = (model.predict(x)).argmax(axis=1)
# ideals = testFULL_Y






# true_positives = 0
# true_negatives = 0
# false_positives = 0
# false_negatives = 0
# cnt_not_fnd = 0
# broke = 0
# pois_pat = 0


# PASS = 0
# PASS_bef = 0
# FAIL = 0
# FAIL_bef = 0

# clean_pass = 0
# clean_fail = 0
# pois_pass = 0
# pois_fail = 0


# for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
#     cnt = cnt + 1
#     image = x[inp_cnt]
#     label = labels[inp_cnt]

#    # inp = image
#    # inc_layer = 'dense'  #layer name, get it from the model summary
#    # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

#     #layer_vals = inc_suff[0]
#     #suff = []



#     img = np.zeros((28,28),dtype=float)
#     for ix in range(0,28):
#       for iy in range(0,28):
#           img[ix][iy] = (float(image[ix][iy][0]))

#     image_array = np.zeros((1,28,28),dtype=float)
#     image_array[0]=img
#     img_mod = np.expand_dims(img,axis=0)


#     inp = image
#     inc_layer = 'dense_1'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals1 = inc_suff[0]
#     suff = []

#     inp = image
#     inc_layer = 'activation_3'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals2 = inc_suff[0]

#     inp = image
#     inc_layer = 'dense_2'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals3 = inc_suff[0]
#     suff = []


#     max_lab7 = False
#     match = -1
#     match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig,support=corr_invs_sel_supp,supp_thres = 50)
#     print("MATCH:", match)
#     if (len(match) == 0):
#       cnt_not_fnd = cnt_not_fnd + 1
#       new_lbl = labels[inp_cnt]
#     else:
#       if (Cneuron_lay[match[0]] == 3):
#         max_lab7 = True
#         if (len(match) > 1):
#           new_lbl = Cneuron_cl[match[1]]
#         else:
#           new_lbl = labels[inp_cnt]
#       else:
#         new_lbl = Cneuron_cl[match[0]]






#     if (new_lbl == ideals[inp_cnt]):
#       PASS = PASS + 1
#       if (labels[inp_cnt] != ideals[inp_cnt]):
#         correct = correct + 1
#         if (max_lab7 == True):
#           pois_pat = pois_pat + 1
#     else:
#       for indx in range(0,len(match)):
#          print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
#       print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
#       FAIL = FAIL + 1
#       if (labels[inp_cnt] == ideals[inp_cnt]):
#         broke = broke + 1
#       #print("BROKE: OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)


# print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

# print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
# print('CORRECTED:', float(correct/cnt)*100)
# print('BROKEN:', float(broke/cnt)*100)
# print('POIS PAT',pois_pat)
# print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )

In [ ]:

# #### GUESSING THE IDEAL LABELS BASED ON PATTERNS
# neuron_ids =[]
# inc_neuron_ids=[]
# neuron_sig = []

# Cneuron_ids =[]
# C_neuron_ids=[]
# Cneuron_sig = []
# Cneuron_cl = []
# Cneuron_lay = []

# corr_invs_sel_supp = []
# corr_invs_sel_neus = []
# corr_invs_sel_sig = []
# corr_invs_sel_labs = []
# corr_invs_sel_lay = []

# for ix in range (0, len(corr_invs_sel_supp1)):
#   #if (corr_invs_sel_supp1[ix] <= 20):
#   #   continue
#   corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])

# # for ix in range (0, len(corr_invs_sel_supp1a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp1a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus1a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig1a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs1a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay1a[ix])

# for ix in range (0, len(corr_invs_sel_supp2)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp2[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus2[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig2[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs2[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay2[ix])

# # for ix in range (0, len(corr_invs_sel_supp2a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp2a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus2a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig2a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs2a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay2a[ix])

# for ix in range (0, len(corr_invs_sel_supp3)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp3[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus3[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig3[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs3[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay3[ix])

# # for ix in range (0, len(corr_invs_sel_supp3a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp3a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus3a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig3a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs3a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay3a[ix])


# for ix in range (0, len(corr_invs_sel_supp)-1):
#     for iy in range(ix +1, len(corr_invs_sel_supp)):
#         if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
#           temp = corr_invs_sel_supp[iy]
#           corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
#           corr_invs_sel_supp[ix] = temp
#           temp = corr_invs_sel_neus[iy]
#           corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
#           corr_invs_sel_neus[ix] = temp
#           temp = corr_invs_sel_sig[iy]
#           corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
#           corr_invs_sel_sig[ix] = temp
#           temp = corr_invs_sel_labs[iy]
#           corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
#           corr_invs_sel_labs[ix] = temp
#           temp = corr_invs_sel_lay[iy]
#           corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
#           corr_invs_sel_lay[ix] = temp


# for indx in range (0, len(corr_invs_sel_neus)):
#       Cneu_ids = []
#       for indx1 in range (0, len(corr_invs_sel_neus[indx])):
#         Cneu_ids.append(-1)
#         Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
#       Cneuron_ids.append(Cneu_ids)
#       C_neuron_ids.append(corr_invs_sel_neus[indx])


# for indx in range (0, len(corr_invs_sel_sig)):
#       Cneuron_sig.append(corr_invs_sel_sig[indx])
#       Cneuron_cl.append(corr_invs_sel_labs[indx])
#       Cneuron_lay.append(corr_invs_sel_lay[indx])

# print("\n CORRECT PATTERNS\n")
# for indx in range (0, len(Cneuron_cl)):
#   print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])


# x = copy.deepcopy(test_X_pois)
# cnt = 0
# correct = 0

# labels = (model.predict(x)).argmax(axis=1)
# ideals = testFULL_Y




# true_positives = 0
# true_negatives = 0
# false_positives = 0
# false_negatives = 0
# cnt_not_fnd = 0
# broke = 0
# pois_pat = 0


# PASS = 0
# PASS_bef = 0
# FAIL = 0
# FAIL_bef = 0

# clean_pass = 0
# clean_fail = 0
# pois_pass = 0
# pois_fail = 0


# for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
#     cnt = cnt + 1
#     image = x[inp_cnt]
#     label = labels[inp_cnt]

#    # inp = image
#    # inc_layer = 'dense'  #layer name, get it from the model summary
#    # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

#     #layer_vals = inc_suff[0]
#     #suff = []



#     img = np.zeros((28,28),dtype=float)
#     for ix in range(0,28):
#       for iy in range(0,28):
#           img[ix][iy] = (float(image[ix][iy][0]))

#     image_array = np.zeros((1,28,28),dtype=float)
#     image_array[0]=img
#     img_mod = np.expand_dims(img,axis=0)


#     inp = image
#     inc_layer = 'dense_1'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals1 = inc_suff[0]
#     suff = []

#     inp = image
#     inc_layer = 'activation_3'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals2 = inc_suff[0]

#     inp = image
#     inc_layer = 'dense_2'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals3 = inc_suff[0]
#     suff = []


#     max_lab7 = False
#     match = -1
#     match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig,support=corr_invs_sel_supp,supp_thres = 50)
#     #print("MATCH:", match)
#     if (len(match) == 0):
#       cnt_not_fnd = cnt_not_fnd + 1
#       new_lbl = labels[inp_cnt]
#     else:
#       labs = []
#       votes = []
#       new_lbl = -1
#       for indx in range(0,len(match)):
#         l = Cneuron_cl[match[indx]]
#         v = 0
#         for indx1 in range(0,len(labs)):
#           if (labs[indx1] == l):
#             votes[indx1] += 1
#             break
#         if (v == 0):
#           labs.append(l)
#           votes.append(v+1)

#       max = -1
#       max_indx = -1
#       for indx in range(0,len(votes)):
#         if (votes[indx] > max):
#           max = votes[indx]
#           max_indx = indx

#       if (max_indx == -1):
#         new_lbl = labels[inp_cnt]
#       else:
#         new_lbl = labs[max_indx]


#     # match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
#     # print("MATCH:", match)
#     # if (len(match) == 0):
#     #   cnt_not_fnd = cnt_not_fnd + 1
#     #   new_lbl = labels[inp_cnt]
#     # else:
#     #   if (Cneuron_lay[match[0]] == 3):
#     #     max_lab7 = True
#     #     if (len(match) > 1):
#     #       new_lbl = Cneuron_cl[match[1]]
#     #     else:
#     #       new_lbl = labels[inp_cnt]
#     #   else:
#     #     new_lbl = Cneuron_cl[match[0]]
#     #   SAME = False
#     #   for indx in range(0,len(match)):
#     #      print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
#     #      if ()



#     if (new_lbl == ideals[inp_cnt]):
#       PASS = PASS + 1
#       if (labels[inp_cnt] != ideals[inp_cnt]):
#         correct = correct + 1
#         if (max_lab7 == True):
#           pois_pat = pois_pat + 1
#     else:
#       for indx in range(0,len(match)):
#           print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
#       print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
#       FAIL = FAIL + 1
#       if (labels[inp_cnt] == ideals[inp_cnt]):
#         broke = broke + 1
#       #print("BROKE: OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)


# print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

# print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
# print('CORRECTED:', float(correct/cnt)*100)
# print('BROKEN:', float(broke/cnt)*100)
# print('POIS PAT',pois_pat)
# print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )


In [ ]:

# #### GUESSING THE IDEAL LABELS BASED ON PATTERNS
# neuron_ids =[]
# inc_neuron_ids=[]
# neuron_sig = []

# Cneuron_ids =[]
# C_neuron_ids=[]
# Cneuron_sig = []
# Cneuron_cl = []
# Cneuron_lay = []

# corr_invs_sel_supp = []
# corr_invs_sel_neus = []
# corr_invs_sel_sig = []
# corr_invs_sel_labs = []
# corr_invs_sel_lay = []

# for ix in range (0, len(corr_invs_sel_supp1)):
#   #if (corr_invs_sel_supp1[ix] <= 20):
#   #   continue
#   corr_invs_sel_supp.append(corr_invs_sel_supp1[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus1[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig1[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs1[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay1[ix])

# # for ix in range (0, len(corr_invs_sel_supp1a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp1a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus1a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig1a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs1a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay1a[ix])

# for ix in range (0, len(corr_invs_sel_supp2)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp2[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus2[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig2[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs2[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay2[ix])

# # for ix in range (0, len(corr_invs_sel_supp2a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp2a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus2a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig2a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs2a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay2a[ix])

# for ix in range (0, len(corr_invs_sel_supp3)):
#   corr_invs_sel_supp.append(corr_invs_sel_supp3[ix])
#   corr_invs_sel_neus.append(corr_invs_sel_neus3[ix])
#   corr_invs_sel_sig.append(corr_invs_sel_sig3[ix])
#   corr_invs_sel_labs.append(corr_invs_sel_labs3[ix])
#   corr_invs_sel_lay.append(corr_invs_sel_lay3[ix])

# # for ix in range (0, len(corr_invs_sel_supp3a)):
# #   corr_invs_sel_supp.append(corr_invs_sel_supp3a[ix])
# #   corr_invs_sel_neus.append(corr_invs_sel_neus3a[ix])
# #   corr_invs_sel_sig.append(corr_invs_sel_sig3a[ix])
# #   corr_invs_sel_labs.append(corr_invs_sel_labs3a[ix])
# #   corr_invs_sel_lay.append(corr_invs_sel_lay3a[ix])


# for ix in range (0, len(corr_invs_sel_supp)-1):
#     for iy in range(ix +1, len(corr_invs_sel_supp)):
#         if (corr_invs_sel_supp[iy] > corr_invs_sel_supp[ix]):
#           temp = corr_invs_sel_supp[iy]
#           corr_invs_sel_supp[iy] = corr_invs_sel_supp[ix]
#           corr_invs_sel_supp[ix] = temp
#           temp = corr_invs_sel_neus[iy]
#           corr_invs_sel_neus[iy] = corr_invs_sel_neus[ix]
#           corr_invs_sel_neus[ix] = temp
#           temp = corr_invs_sel_sig[iy]
#           corr_invs_sel_sig[iy] = corr_invs_sel_sig[ix]
#           corr_invs_sel_sig[ix] = temp
#           temp = corr_invs_sel_labs[iy]
#           corr_invs_sel_labs[iy] = corr_invs_sel_labs[ix]
#           corr_invs_sel_labs[ix] = temp
#           temp = corr_invs_sel_lay[iy]
#           corr_invs_sel_lay[iy] = corr_invs_sel_lay[ix]
#           corr_invs_sel_lay[ix] = temp


# for indx in range (0, len(corr_invs_sel_neus)):
#       Cneu_ids = []
#       for indx1 in range (0, len(corr_invs_sel_neus[indx])):
#         Cneu_ids.append(-1)
#         Cneu_ids.append(corr_invs_sel_neus[indx][indx1])
#       Cneuron_ids.append(Cneu_ids)
#       C_neuron_ids.append(corr_invs_sel_neus[indx])


# for indx in range (0, len(corr_invs_sel_sig)):
#       Cneuron_sig.append(corr_invs_sel_sig[indx])
#       Cneuron_cl.append(corr_invs_sel_labs[indx])
#       Cneuron_lay.append(corr_invs_sel_lay[indx])

# print("\n CORRECT PATTERNS\n")
# for indx in range (0, len(Cneuron_cl)):
#   print(Cneuron_cl[indx],corr_invs_sel_supp[indx],Cneuron_lay[indx],C_neuron_ids[indx],Cneuron_sig[indx])



# gen_data = test_X_pois
# gen_labels = test_Y
# actuals_gen = (model.predict(gen_data)).argmax(axis=1)

# x = copy.deepcopy(gen_data)
# cnt = 0
# correct = 0

# labels = actuals_gen #actuals_gen
# ideals = gen_labels #gen_labels




# true_positives = 0
# true_negatives = 0
# false_positives = 0
# false_negatives = 0
# cnt_not_fnd = 0
# broke = 0
# pois_pat = 0


# PASS = 0
# PASS_bef = 0
# FAIL = 0
# FAIL_bef = 0

# clean_pass = 0
# clean_fail = 0
# pois_pass = 0
# pois_fail = 0


# for inp_cnt in range(0,len(x)): # x is the list of images in the dataset used to extract the mis-classification pattern
#     cnt = cnt + 1
#     image = x[inp_cnt]
#     label = labels[inp_cnt]

#    # inp = image
#    # inc_layer = 'dense'  #layer name, get it from the model summary
#    # inc_suff = fingerprint_suffix_vals(inp, inc_layer)

#     #layer_vals = inc_suff[0]
#     #suff = []



#     img = np.zeros((28,28),dtype=float)
#     for ix in range(0,28):
#       for iy in range(0,28):
#           img[ix][iy] = (float(image[ix][iy][0]))

#     image_array = np.zeros((1,28,28),dtype=float)
#     image_array[0]=img
#     img_mod = np.expand_dims(img,axis=0)


#     inp = image
#     inc_layer = 'dense_1'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals1 = inc_suff[0]
#     suff = []

#     inp = image
#     inc_layer = 'activation_3'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals2 = inc_suff[0]

#     inp = image
#     inc_layer = 'dense_2'  #layer name, get it from the model summary
#     inc_suff = fingerprint_suffix_vals(inp, inc_layer)
#     layer_vals3 = inc_suff[0]
#     suff = []


#     max_lab7 = False
#     match = -1
#     Same = True
#     match = check_pattern_inter_lays(layer_vals1,layer_vals2,layer_vals3,Cneuron_lay,Cneuron_ids,Cneuron_sig)
#     print("MATCH:", match)
#     if (len(match) == 0):
#       cnt_not_fnd = cnt_not_fnd + 1
#       new_lbl = labels[inp_cnt]
#     else:
#       if (Cneuron_lay[match[0]] == 3):
#         max_lab7 = True
#         if (len(match) > 1):
#           new_lbl = Cneuron_cl[match[1]]
#         else:
#           new_lbl = labels[inp_cnt]
#       else:
#         new_lbl = Cneuron_cl[match[0]]

#       prev_cls = -1
#       for indx in range(0,len(match)):
#          print("LAYER,LABEL,SUPP", Cneuron_lay[match[indx]],Cneuron_cl[match[indx]], corr_invs_sel_supp[match[indx]])
#          if (Cneuron_cl[match[indx]] != prev_cls and (Same == True)):
#            Same = False
#            prev_cls = Cneuron_cl[match[indx]]





#     if (new_lbl == ideals[inp_cnt]):
#       PASS = PASS + 1
#       if (labels[inp_cnt] != ideals[inp_cnt]):
#   #      print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
#         correct = correct + 1
#         if (max_lab7 == True):
#           pois_pat = pois_pat + 1
#     else:
#       if (Same == False):
#         print("OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)
#       FAIL = FAIL + 1
#       if (labels[inp_cnt] == ideals[inp_cnt]):
#         broke = broke + 1
#       #print("BROKE: OLD LABEL, IDEAL LABEL, MATCH,NEW LABEL:", labels[inp_cnt], ideals[inp_cnt],match,new_lbl)


# print("\n DISTILLATION BASED ON PATTERNS FOR IDEAL LABEL:")

# print('TOTAL GUESSED BASED ON PATTERNs :',float((cnt - cnt_not_fnd)/cnt)*100)
# print('CORRECTED:', float(correct/cnt)*100)
# print('BROKEN:', float(broke/cnt)*100)
# print('POIS PAT',pois_pat)
# print("PASS:", PASS, ", FAIL:", FAIL, ", OVERALL ACCURACY with monitor:", (PASS/(PASS+FAIL)) * 100 )
